In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
import random
from sklearn.metrics import mean_absolute_error, mean_squared_error
from keras.layers import Conv1D, MaxPooling1D, Bidirectional, LSTM, Dropout, Dense, Layer
from sklearn.preprocessing import StandardScaler
import warnings
from keras.callbacks import EarlyStopping
from keras.layers import Conv1D, MaxPooling1D, Dense, Flatten
from keras.optimizers import Adam
import keras.backend as K
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', '{:.0f}'.format)
import os
import tensorflow as tf
from datetime import datetime

In [ ]:
def results(a,b,c,d,e,f,g):
    current_time = datetime.now()
    data = {
        'model' : [g],
        'sim' : [a],
        'mae' : [b],
        'rmse' : [c],
        'fsd' : [d],
        'R' : [e],
        'NSE': [f],
        'time' : [current_time]
    }
    df = pd.DataFrame(data)
    with open('results_combine.csv', 'a', newline='') as f:
        if os.path.isfile('results_combine.csv'):
            df.to_csv('results_combine.csv', mode='a', header=False, index=False)
        else:
            df.to_csv('results_combine.csv', index=False)

def to_df(data_list):
    X_df = [i[:-1] for i in data_list]
    y_df = [i[-1] for i in data_list]
    transposed_lists = [list(x) for x in zip(*X_df)]

    df_list = pd.DataFrame({f'Column{i+1}': lst for i, lst in enumerate(transposed_lists)})
    df_list['Target'] = y_df
    return df_list

def transform_to_multivariate(data, T):
    M = []
    for i in range(len(data) - T):
        row = data[i:i + T + 1]
        M.append(row)
    return np.array(M)

def calculate_similarity(value_lst_after, value_lst_before):
        T = len(value_lst_after)  # Number of missing values
        similarity_sum = 0

        for i in range(T):
            yi = value_lst_after[i]
            xi = value_lst_before[i]
            similarity_sum += 1 / (1 + abs(yi - xi) / (max(value_lst_before) - min(value_lst_before)))

        similarity = similarity_sum / T
        return similarity

def calculate_MAE(value_lst_missing, value_lst_after):
        return mean_absolute_error(value_lst_missing, value_lst_after)

def calculate_RMSE(value_lst_missing, value_lst_after):
    return np.sqrt(mean_squared_error(value_lst_missing, value_lst_after))

def calculate_FB(value_lst_missing, value_lst_after):
    return 2 * abs((np.mean(value_lst_after) - np.mean(value_lst_missing)) / (np.mean(value_lst_after) + np.mean(value_lst_missing)))

def calculate_fsd(value_lst_missing, value_lst_after):
    std_dev_Y = np.std(value_lst_after)
    std_dev_X = np.std(value_lst_missing)

    if std_dev_X == 0:
        return None
    
    fsd = 2 * abs((std_dev_Y - std_dev_X) / (std_dev_X + std_dev_Y))
    
    return fsd

def calculate_r_score(value_lst_missing, value_lst_after):

    correlation_matrix = np.corrcoef(value_lst_missing, value_lst_after)
    r_score = correlation_matrix[0, 1]
    return r_score

def calculate_nse(value_lst_missing, value_lst_after):

    value_lst_missing = np.array(value_lst_missing)
    value_lst_after = np.array(value_lst_after)

    numerator = np.sum((value_lst_missing - value_lst_after)**2)
    denominator = np.sum((value_lst_missing - np.mean(value_lst_missing))**2)

    nse = 1 - (numerator / denominator)
    
    return nse

def calculate_metrics_for_combine(value_lst_after,name_model):
    
    df_before_missing = pd.read_csv('waterlevel.csv')
    value_lst_missing = df_before_missing['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap]


    similarity_score = calculate_similarity(value_lst_after, value_lst_missing)
    MAE_score = calculate_MAE(value_lst_missing, value_lst_after)
    RMSE_score = calculate_RMSE(value_lst_missing, value_lst_after)
    FSD_score = calculate_fsd(value_lst_missing, value_lst_after)
    R_score = calculate_r_score(value_lst_missing, value_lst_after)
    NSE_score = calculate_nse(value_lst_missing, value_lst_after)
    
    sim_lst_combine.append(similarity_score)
    mae_lst_combine.append(MAE_score)
    rmse_lst_combine.append(RMSE_score)
    fsd_lst_combine.append(FSD_score)
    r_lst_combine.append(R_score)
    nse_lst_combine.append(NSE_score)

    
    print('\nOri_data:', value_lst_missing)
    print('\nvalue_data:', value_lst_after)
    print('\nSimilarity_score:', similarity_score)
    print('\nMean Absolute Error (MAE):', MAE_score)
    print('\nRoot Mean Squared Error (RMSE):', RMSE_score)
    print('\nFraction of Standard Deviation Score:', FSD_score)
    print('\nR score:', R_score)
    print('\nThe Nash Sutcliffe efficiency (NSE):', NSE_score)

    results(similarity_score, MAE_score, RMSE_score, FSD_score, R_score, NSE_score,name_model)

def create_continuous_missing_values(dataframe, column_name, num_missing_values):
    modified_df = dataframe.copy()
    
    if len(dataframe) > num_missing_values:
        random_index = random.randint(0, len(dataframe) - num_missing_values)
        modified_df.loc[random_index:random_index + num_missing_values - 1, column_name] = np.nan
    else:
        print("Error: The number of missing values requested exceeds the DataFrame's capacity.")
    return modified_df


sim_lst_combine = []
mae_lst_combine = []
rmse_lst_combine = []
fsd_lst_combine = []
r_lst_combine = []
nse_lst_combine = []

scaler = StandardScaler()

In [ ]:
# original_df_path = 'waterlevel.csv'
# try:
#     original_df = pd.read_csv(original_df_path)
    
#     for i in range(0, 12):
#         modified_df = create_continuous_missing_values(original_df, 'Waterlevel', 48)
#         modified_df.to_csv(f'waterlevel_missing_test_{i}.csv', index=False)
        
#         print(f'waterlevel_missing_test_{i}.csv saved with continuous missing values.')

# except FileNotFoundError:
#     print(f"Failed to find '{original_df_path}'. Please check the file path and try again.")

In [ ]:
def model_combine(X_train):
    combine = tf.keras.models.Sequential()

    combine.add(Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
    combine.add(MaxPooling1D(pool_size=2))
    combine.add(Dropout(0.15))
    
    combine.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
    combine.add(Dropout(0.15))
    
    combine.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
    combine.add(Dropout(0.15))
    
    combine.add(Bidirectional(LSTM(units=64, return_sequences=True)))
    combine.add(Dropout(0.15))

    class Attention(Layer):
        def __init__(self, **kwargs):
            super(Attention, self).__init__(**kwargs)
    
        def build(self, input_shape):
            self.W = self.add_weight(name='attention_weight', shape=(input_shape[-1], 1), 
                                     initializer='random_normal', trainable=True)
            self.b = self.add_weight(name='attention_bias', shape=(input_shape[1], 1), 
                                     initializer='zeros', trainable=True)        
            super(Attention, self).build(input_shape)
     
        def call(self, x):
            e = K.tanh(K.dot(x, self.W) + self.b)
            e = K.squeeze(e, axis=-1)
            alpha = K.softmax(e)
            alpha = K.expand_dims(alpha, axis=-1)
            context = x * alpha
            context = K.sum(context, axis=1)
            return context

    combine.add(Attention())
    
    combine.add(Dense(units=64, activation='relu'))
    
    combine.add(Dense(units=1))
    
    combine.compile(optimizer=Adam(), loss="mean_squared_error", metrics=['mae'])
    
    return combine

### epochs=200, batch_size=16, callbacks (patience=30), validation_split=0.2

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# tf.debugging.set_log_device_placement(True)

early_stopping = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)

weights_folder = 'model_weights'
os.makedirs(weights_folder, exist_ok=True)

for location in range(4,12):
    
    filename = f'waterlevel_missing/waterlevel_missing_{location}.csv'
    df = pd.read_csv(filename)
    
    #### Check size of missing value
    size_of_gap = df['Waterlevel'].isna().sum()

    data = df['Waterlevel'].values.tolist()

    nan_index = None
    for j, value in enumerate(data):
        if value != value:  # Check if the value is NaN
            nan_index = j
            break
        
    df_check_before = data[:(3*size_of_gap)+1]
    df_check_after = data[::-1][:3*size_of_gap+1]
    df_miss = data[nan_index:nan_index + size_of_gap]

    last_data = data[:nan_index]
    first_data = data[nan_index+size_of_gap:][::-1]

    # check if missing values is in the first 3 x T data original
    if all(value in df_check_before for value in df_miss):
        print('\nAll values in df_miss is in the first !!!')

        first_value_combine = transform_to_multivariate(first_data, size_of_gap)
                
        df_list = to_df(first_value_combine)
        

        X = np.array(df_list.iloc[:, :-1])
        X = scaler.fit_transform(X)
        
        y = np.array(df_list.iloc[:, -1])
        
        model = model_combine(X)
        
        weights_path = os.path.join(weights_folder, f'model_weights_location_{location}.h5')
        if os.path.exists(weights_path):
            model.load_weights(weights_path)
            print(f"Loaded weights for location {location}")
        else:
            model.fit(X, y, epochs=200, batch_size=16, callbacks=[early_stopping], validation_split=0.2)
            model.save_weights(weights_path)
            print(f"Saved weights for location {location}")
            
            
        # model.fit(X, y, epochs=200, batch_size=16, callbacks=[early_stopping], validation_split=0.2)

        data_test = df.values.tolist()[nan_index : nan_index + 2 * size_of_gap][::-1]
        data_test = np.concatenate(data_test).ravel()   

        results_first_combine = []
        for i in range(len(data_test)//2):
            data_first = data_test[i:i+1+size_of_gap]

            data_first[size_of_gap] = model.predict(scaler.transform(np.array(data_first[:size_of_gap]).reshape(1,-1)))

            results_first_combine.append(data_first[size_of_gap])
            
        ###################################################################
        print('\n', 'result of combine only (first):')                        #  
        calculate_metrics_for_combine(results_first_combine,'results_combine (FIRST)')#
        print('\n')                                                       #
        ###################################################################
        
        df = pd.read_csv('Impute_misvalues_hungyen.csv')

        plt.figure(figsize=(10, 6))
        plt.plot(df['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap], label='Water Level')
        plt.plot(results_first_combine, label='Predicted Value', linestyle='-')
        plt.title(f'Missing {location}, epochs=200, batch_size=16, patience=30, validation_split=0.2')
        plt.xlabel('Index')
        plt.ylabel('Value')
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        folder_path = 'P:\SU24\Imputation\HY_git\\128,256,512,64,64'
        filename = f'128,256,512,64,64, 2nd, Missing {location}, epochs=200, batch_size=16, patience=30, validation_split=0.2.png'
        full_path = os.path.join(folder_path, filename)
        plt.savefig(full_path)
        plt.show()
        
    elif all(value in df_check_after for value in df_miss):
        print('\nAll values in df_miss is in the last !!!')
        
        last_value_combine = transform_to_multivariate(last_data, size_of_gap)
                
        df_list = to_df(last_value_combine)

        X = np.array(df_list.iloc[:, :-1])
        X = scaler.fit_transform(X)
        
        y = np.array(df_list.iloc[:, -1])
        
        model = model_combine(X)
        
        weights_path = os.path.join(weights_folder, f'model_weights_location_{location}.h5')
        if os.path.exists(weights_path):
            model.load_weights(weights_path)
            print(f"Loaded weights for location {location}")
        else:
            model.fit(X, y, epochs=200, batch_size=16, callbacks=[early_stopping], validation_split=0.2)
            model.save_weights(weights_path)
            print(f"Saved weights for location {location}")
            
            
        # model.fit(X, y, epochs=200, batch_size=16, callbacks=[early_stopping], validation_split=0.2)

        data_test = df.values.tolist()[nan_index - size_of_gap : nan_index + size_of_gap]
        data_test = np.concatenate(data_test).ravel()   

        results_last_combine = []
        for i in range(len(data_test)//2):
            data_last = data_test[i:i+1+size_of_gap]

            data_last[size_of_gap] = model.predict(scaler.transform(np.array(data_last[:size_of_gap]).reshape(1,-1)))

            results_last_combine.append(data_last[size_of_gap])
            
        #################################################################
        print('\n', 'result of combine only (last):')                       #
        calculate_metrics_for_combine(results_last_combine,'results_combine (LAST)')#
        print('\n')                                                     #
        # #################################################################
        
        df = pd.read_csv('Impute_misvalues_hungyen.csv')

        plt.figure(figsize=(10, 6))
        plt.plot(df['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap], label='Water Level')
        plt.plot(results_last_combine, label='Predicted Value', linestyle='-')
        plt.title(f'Missing {location}, epochs=200, batch_size=16, patience=30, validation_split=0.2')
        plt.xlabel('Index')
        plt.ylabel('Value')
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        folder_path = 'P:\SU24\Imputation\HY_git\\128,256,512,64,64'
        filename = f'128,256,512,64,64, 2nd, Missing {location}, epochs=200, batch_size=16, patience=30, validation_split=0.2.png'
        full_path = os.path.join(folder_path, filename)
        plt.savefig(full_path)
        plt.show()
        
    else:
        Da = data[nan_index + size_of_gap:][::-1]

        MDa = transform_to_multivariate(Da, size_of_gap)

        df_MDa = to_df(MDa)

        X_MDa_train = np.array(df_MDa.iloc[:, :-1])
        X_MDa_train = scaler.fit_transform(X_MDa_train)
        
        y_MDa_train = np.array(df_MDa.iloc[:, -1])

        model_MDa = model_combine(X_MDa_train)
        
        weights_path_after = os.path.join(weights_folder, f'model_weights_after_{location}.h5')
        if os.path.exists(weights_path_after):
            model_MDa.load_weights(weights_path_after)
            print(f"Loaded weights for after the gap at location {location}")
        else:
            model_MDa.fit(X_MDa_train, y_MDa_train, epochs=200, batch_size=16, callbacks=[early_stopping], validation_split=0.2)
            model_MDa.save_weights(weights_path_after)
            print(f"Saved weights for after the gap at location {location}")
            
            
        # model_MDa.fit(X_MDa_train, y_MDa_train, epochs=200, batch_size=16, callbacks=[early_stopping], validation_split=0.2)
        
        data_test_after = df.values.tolist()[nan_index:nan_index + 2 * size_of_gap ][::-1]
        data_test_after = np.concatenate(data_test_after).ravel()   

        value_lst_after = []
        for j in range(len(data_test_after)//2):
            data_after = data_test_after[j:j+1+size_of_gap]
            
            data_after[size_of_gap] = model_MDa.predict(scaler.transform(np.array(data_after[:size_of_gap]).reshape(1,-1)))
            
            value_lst_after.append(data_after[size_of_gap])

        Db = data[:nan_index]

        MDb = transform_to_multivariate(Db, size_of_gap)  

        df_MDb = to_df(MDb)

        X_MDb_train = np.array(df_MDb.iloc[:, :-1])
        X_MDb_train = scaler.fit_transform(X_MDb_train)
        
        y_MDb_train = np.array(df_MDb.iloc[:, -1])

        model_MDb = model_combine(X_MDb_train)
        
        weights_path_before = os.path.join(weights_folder, f'model_weights_before_{location}.h5')
        if os.path.exists(weights_path_before):
            model_MDb.load_weights(weights_path_before)
            print(f"Loaded weights for before the gap at location {location}")
        else:
            model_MDb.fit(X_MDb_train, y_MDb_train, epochs=200, batch_size=16, callbacks=[early_stopping], validation_split=0.2)
            model_MDb.save_weights(weights_path_before)
            print(f"Saved weights for after the gap at location {location}")
            
        # model_MDb.fit(X_MDb_train, y_MDb_train, epochs=200, batch_size=16, callbacks=[early_stopping], validation_split=0.2)

        data_test_before = df.values.tolist()[nan_index - size_of_gap : nan_index + size_of_gap]
        data_test_before = np.concatenate(data_test_before).ravel()   

        value_lst_before = []
        for i in range(len(data_test_before)//2):
            data_before = data_test_before[i:i+1+size_of_gap]
            
            data_before[size_of_gap] = model_MDb.predict(scaler.transform(np.array(data_before[:size_of_gap]).reshape(1,-1)))

            value_lst_before.append(data_before[size_of_gap])

        #############################################################################
        print('\n', 'result of combine only: ')                                         #
        results_combine =  [(x + y)/2 for x,y in zip(value_lst_before, value_lst_after)]#
        calculate_metrics_for_combine(results_combine,'results_combine (MIDDLE)')               #
        print('\n')                                                                 #
        #############################################################################
        
        df = pd.read_csv('Impute_misvalues_hungyen.csv')

        plt.figure(figsize=(10, 6))
        plt.plot(df['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap], label='Water Level')
        plt.plot(results_combine, label='Predicted Value', linestyle='-')
        plt.title(f'Missing {location}, epochs=200, batch_size=16, patience=30, validation_split=0.2')
        plt.xlabel('Index')
        plt.ylabel('Value')
        plt.grid(True)
        plt.legend()
        plt.tight_layout()   
        folder_path = 'P:\SU24\Imputation\HY_git\\128,256,512,64,64'
        filename = f'128,256,512,64,64, 2nd, Missing {location}, epochs=200, batch_size=16, patience=30, validation_split=0.2.png'
        full_path = os.path.join(folder_path, filename)
        plt.savefig(full_path)
        plt.show()
    
    
current_time = datetime.now()

print('\n')
print('\nMean of Similarity combine: ',np.mean(sim_lst_combine))
print('\nMean of Mean Absoulute Error combine :',np.mean(mae_lst_combine))
print('\nMean of Root Mean Squared Error combine: ',np.mean(rmse_lst_combine))
print('\nMean of Fraction of Standard Deviation combine: ',np.mean(fsd_lst_combine)) 
print('\nMean of R-score combine: ', np.mean(r_lst_combine))
print('\nMean of the Nash Sutcliffe efficiency (NSE): ', np.mean(nse_lst_combine))
print('\n')

In [6]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# tf.debugging.set_log_device_placement(True)

early_stopping = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)

weights_folder = 'model_weights'
os.makedirs(weights_folder, exist_ok=True)

for location in range(0,12):
    
    filename = f'waterlevel_missing/waterlevel_missing_{location}.csv'
    df = pd.read_csv(filename)
    
    #### Check size of missing value
    size_of_gap = df['Waterlevel'].isna().sum()

    data = df['Waterlevel'].values.tolist()

    nan_index = None
    for j, value in enumerate(data):
        if value != value:  # Check if the value is NaN
            nan_index = j
            break
        
    df_check_before = data[:(3*size_of_gap)+1]
    df_check_after = data[::-1][:3*size_of_gap+1]
    df_miss = data[nan_index:nan_index + size_of_gap]

    last_data = data[:nan_index]
    first_data = data[nan_index+size_of_gap:][::-1]

    # check if missing values is in the first 3 x T data original
    if all(value in df_check_before for value in df_miss):
        print('\nAll values in df_miss is in the first !!!')

        first_value_combine = transform_to_multivariate(first_data, size_of_gap)
                
        df_list = to_df(first_value_combine)
        

        X = np.array(df_list.iloc[:, :-1])
        X = scaler.fit_transform(X)
        
        y = np.array(df_list.iloc[:, -1])
        
        model = model_combine(X)
        
        weights_path = os.path.join(weights_folder, f'model_weights_location_{location}.h5')

        new_weights_path = os.path.join(weights_folder, f'model_weights_location_{location}_1.h5')
        model.load_weights(weights_path)
        
        model.fit(X, y, epochs=200, batch_size=16, callbacks=[early_stopping], validation_split=0.2)
        model.save_weights(new_weights_path)
            
            
        data_test = df.values.tolist()[nan_index : nan_index + 2 * size_of_gap][::-1]
        data_test = np.concatenate(data_test).ravel()   

        results_first_combine = []
        for i in range(len(data_test)//2):
            data_first = data_test[i:i+1+size_of_gap]

            data_first[size_of_gap] = model.predict(scaler.transform(np.array(data_first[:size_of_gap]).reshape(1,-1)))

            results_first_combine.append(data_first[size_of_gap])
            
        ###################################################################
        print('\n', 'result of combine only (first):')                        #  
        calculate_metrics_for_combine(results_first_combine,'results_combine (FIRST)')#
        print('\n')                                                       #
        ###################################################################
        
        df = pd.read_csv('Impute_misvalues_hungyen.csv')

        plt.figure(figsize=(10, 6))
        plt.plot(df['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap], label='Water Level')
        plt.plot(results_first_combine, label='Predicted Value', linestyle='-')
        plt.title(f'Missing {location}, epochs=200, batch_size=16, patience=30, validation_split=0.2')
        plt.xlabel('Index')
        plt.ylabel('Value')
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        folder_path = 'P:\SU24\Imputation\HY_git\\128,256,512,64,64'
        filename = f'128,256,512,64,64, 3nd, Missing {location}, epochs=200, batch_size=16, patience=30, validation_split=0.2.png'
        full_path = os.path.join(folder_path, filename)
        plt.savefig(full_path)
        plt.show()
        
    elif all(value in df_check_after for value in df_miss):
        print('\nAll values in df_miss is in the last !!!')
        
        last_value_combine = transform_to_multivariate(last_data, size_of_gap)
                
        df_list = to_df(last_value_combine)

        X = np.array(df_list.iloc[:, :-1])
        X = scaler.fit_transform(X)
        
        y = np.array(df_list.iloc[:, -1])
        
        model = model_combine(X)
        
        weights_path = os.path.join(weights_folder, f'model_weights_location_{location}.h5')

        new_weights_path = os.path.join(weights_folder, f'model_weights_location_{location}_1.h5')
        model.load_weights(weights_path)
        
        model.fit(X, y, epochs=200, batch_size=16, callbacks=[early_stopping], validation_split=0.2)
        model.save_weights(new_weights_path)


        data_test = df.values.tolist()[nan_index - size_of_gap : nan_index + size_of_gap]
        data_test = np.concatenate(data_test).ravel()   

        results_last_combine = []
        for i in range(len(data_test)//2):
            data_last = data_test[i:i+1+size_of_gap]

            data_last[size_of_gap] = model.predict(scaler.transform(np.array(data_last[:size_of_gap]).reshape(1,-1)))

            results_last_combine.append(data_last[size_of_gap])
            
        #################################################################
        print('\n', 'result of combine only (last):')                       #
        calculate_metrics_for_combine(results_last_combine,'results_combine (LAST)')#
        print('\n')                                                     #
        # #################################################################
        
        df = pd.read_csv('Impute_misvalues_hungyen.csv')

        plt.figure(figsize=(10, 6))
        plt.plot(df['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap], label='Water Level')
        plt.plot(results_last_combine, label='Predicted Value', linestyle='-')
        plt.title(f'Missing {location}, epochs=200, batch_size=16, patience=30, validation_split=0.2')
        plt.xlabel('Index')
        plt.ylabel('Value')
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        folder_path = 'P:\SU24\Imputation\HY_git\\128,256,512,64,64'
        filename = f'128,256,512,64,64, 3nd, Missing {location}, epochs=200, batch_size=16, patience=30, validation_split=0.2.png'
        full_path = os.path.join(folder_path, filename)
        plt.savefig(full_path)
        plt.show()
        
    else:
        Da = data[nan_index + size_of_gap:][::-1]

        MDa = transform_to_multivariate(Da, size_of_gap)

        df_MDa = to_df(MDa)

        X_MDa_train = np.array(df_MDa.iloc[:, :-1])
        X_MDa_train = scaler.fit_transform(X_MDa_train)
        
        y_MDa_train = np.array(df_MDa.iloc[:, -1])

        model_MDa = model_combine(X_MDa_train)
            
        weights_path_after = os.path.join(weights_folder, f'model_weights_after_{location}.h5')

        new_weights_path_after = os.path.join(weights_folder, f'model_weights_after_{location}_1.h5')
        model_MDa.load_weights(weights_path_after)
        
        model_MDa.fit(X_MDa_train, y_MDa_train, epochs=200, batch_size=16, callbacks=[early_stopping], validation_split=0.2)
        model_MDa.save_weights(new_weights_path_after)
        
        data_test_after = df.values.tolist()[nan_index:nan_index + 2 * size_of_gap ][::-1]
        data_test_after = np.concatenate(data_test_after).ravel()   

        value_lst_after = []
        for j in range(len(data_test_after)//2):
            data_after = data_test_after[j:j+1+size_of_gap]
            
            data_after[size_of_gap] = model_MDa.predict(scaler.transform(np.array(data_after[:size_of_gap]).reshape(1,-1)))
            
            value_lst_after.append(data_after[size_of_gap])

        Db = data[:nan_index]

        MDb = transform_to_multivariate(Db, size_of_gap)  

        df_MDb = to_df(MDb)

        X_MDb_train = np.array(df_MDb.iloc[:, :-1])
        X_MDb_train = scaler.fit_transform(X_MDb_train)
        
        y_MDb_train = np.array(df_MDb.iloc[:, -1])

        model_MDb = model_combine(X_MDb_train)
        
        weights_path_before = os.path.join(weights_folder, f'model_weights_before_{location}.h5')

        new_weights_path_before = os.path.join(weights_folder, f'model_weights_before_{location}_1.h5')
        model_MDb.load_weights(weights_path_before)
        
        model_MDb.fit(X_MDb_train, y_MDb_train, epochs=200, batch_size=16, callbacks=[early_stopping], validation_split=0.2)
        model_MDb.save_weights(new_weights_path_before)

        data_test_before = df.values.tolist()[nan_index - size_of_gap : nan_index + size_of_gap]
        data_test_before = np.concatenate(data_test_before).ravel()   

        value_lst_before = []
        for i in range(len(data_test_before)//2):
            data_before = data_test_before[i:i+1+size_of_gap]
            
            data_before[size_of_gap] = model_MDb.predict(scaler.transform(np.array(data_before[:size_of_gap]).reshape(1,-1)))

            value_lst_before.append(data_before[size_of_gap])

        #############################################################################
        print('\n', 'result of combine only: ')                                         #
        results_combine =  [(x + y)/2 for x,y in zip(value_lst_before, value_lst_after)]#
        calculate_metrics_for_combine(results_combine,'results_combine (MIDDLE)')               #
        print('\n')                                                                 #
        #############################################################################
        
        df = pd.read_csv('Impute_misvalues_hungyen.csv')

        plt.figure(figsize=(10, 6))
        plt.plot(df['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap], label='Water Level')
        plt.plot(results_combine, label='Predicted Value', linestyle='-')
        plt.title(f'Missing {location}, epochs=200, batch_size=16, patience=30, validation_split=0.2')
        plt.xlabel('Index')
        plt.ylabel('Value')
        plt.grid(True)
        plt.legend()
        plt.tight_layout()   
        folder_path = 'P:\SU24\Imputation\HY_git\\128,256,512,64,64'
        filename = f'128,256,512,64,64, 3nd, Missing {location}, epochs=200, batch_size=16, patience=30, validation_split=0.2.png'
        full_path = os.path.join(folder_path, filename)
        plt.savefig(full_path)
        plt.show()
    
    
current_time = datetime.now()

print('\n')
print('\nMean of Similarity combine: ',np.mean(sim_lst_combine))
print('\nMean of Mean Absoulute Error combine :',np.mean(mae_lst_combine))
print('\nMean of Root Mean Squared Error combine: ',np.mean(rmse_lst_combine))
print('\nMean of Fraction of Standard Deviation combine: ',np.mean(fsd_lst_combine)) 
print('\nMean of R-score combine: ', np.mean(r_lst_combine))
print('\nMean of the Nash Sutcliffe efficiency (NSE): ', np.mean(nse_lst_combine))
print('\n')

Num GPUs Available:  1

All values in df_miss is in the first !!!
Epoch 1/200
3198/3198 [==============================] - 26s 8ms/step - loss: 21.7620 - mae: 3.4140 - val_loss: 237.3962 - val_mae: 8.1279
Epoch 2/200
3198/3198 [==============================] - 27s 8ms/step - loss: 22.1513 - mae: 3.4283 - val_loss: 190.6691 - val_mae: 7.5576
Epoch 3/200
3198/3198 [==============================] - 27s 8ms/step - loss: 21.7473 - mae: 3.4071 - val_loss: 186.3320 - val_mae: 6.8875
Epoch 4/200
3198/3198 [==============================] - 25s 8ms/step - loss: 21.5798 - mae: 3.3966 - val_loss: 142.3781 - val_mae: 6.4930
Epoch 5/200
1755/3198 [===============>..............] - ETA: 10s - loss: 20.8047 - mae: 3.3529

KeyboardInterrupt: 

In [ ]:
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# # tf.debugging.set_log_device_placement(True)

# early_stopping = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)


# for location in range(0,12):
    
#     filename = f'waterlevel_missing/waterlevel_missing_{location}.csv'
#     df = pd.read_csv(filename)
    
#     #### Check size of missing value
#     size_of_gap = df['Waterlevel'].isna().sum()

#     data = df['Waterlevel'].values.tolist()

#     nan_index = None
#     for j, value in enumerate(data):
#         if value != value:  # Check if the value is NaN
#             nan_index = j
#             break
        
#     df_check_before = data[:(3*size_of_gap)+1]
#     df_check_after = data[::-1][:3*size_of_gap+1]
#     df_miss = data[nan_index:nan_index + size_of_gap]

#     last_data = data[:nan_index]
#     first_data = data[nan_index+size_of_gap:][::-1]

#     # check if missing values is in the first 3 x T data original
#     if all(value in df_check_before for value in df_miss):
#         print('\nAll values in df_miss is in the first !!!')

#         first_value_combine = transform_to_multivariate(first_data, size_of_gap)
                
#         df_list = to_df(first_value_combine)
        

#         X = np.array(df_list.iloc[:, :-1])
#         X = scaler.fit_transform(X)
        
#         y = np.array(df_list.iloc[:, -1])
        
#         model = model_combine(X)
#         model.fit(X, y, epochs=200, batch_size=16, callbacks=[early_stopping], validation_split=0.2)

#         data_test = df.values.tolist()[nan_index : nan_index + 2 * size_of_gap][::-1]
#         data_test = np.concatenate(data_test).ravel()   

#         results_first_combine = []
#         for i in range(len(data_test)//2):
#             data_first = data_test[i:i+1+size_of_gap]

#             data_first[size_of_gap] = model.predict(scaler.transform(np.array(data_first[:size_of_gap]).reshape(1,-1)))

#             results_first_combine.append(data_first[size_of_gap])
            
#         ###################################################################
#         print('\n', 'result of combine only (first):')                        #  
#         calculate_metrics_for_combine(results_first_combine,'results_combine (FIRST)')#
#         print('\n')                                                       #
#         ###################################################################
        
#         df = pd.read_csv('Impute_misvalues_hungyen.csv')

#         plt.figure(figsize=(10, 6))
#         plt.plot(df['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap], label='Water Level')
#         plt.plot(results_first_combine, label='Predicted Value', linestyle='-')
#         plt.title(f'Missing {location}, epochs=200, batch_size=16, patience=30, validation_split=0.2')
#         plt.xlabel('Index')
#         plt.ylabel('Value')
#         plt.grid(True)
#         plt.legend()
#         plt.tight_layout()
#         folder_path = 'P:\SU24\Imputation\HY_git\\128,256,512,128,128'
#         filename = f'128,256,512,128,128, Missing {location}, epochs=200, batch_size=16, patience=30, validation_split=0.2.png'
#         full_path = os.path.join(folder_path, filename)
#         plt.savefig(full_path)
#         plt.show()
        
#     elif all(value in df_check_after for value in df_miss):
#         print('\nAll values in df_miss is in the last !!!')
        
#         last_value_combine = transform_to_multivariate(last_data, size_of_gap)
                
#         df_list = to_df(last_value_combine)

#         X = np.array(df_list.iloc[:, :-1])
#         X = scaler.fit_transform(X)
        
#         y = np.array(df_list.iloc[:, -1])
        
#         model = model_combine(X)
#         model.fit(X, y, epochs=200, batch_size=16, callbacks=[early_stopping], validation_split=0.2)

#         data_test = df.values.tolist()[nan_index - size_of_gap : nan_index + size_of_gap]
#         data_test = np.concatenate(data_test).ravel()   

#         results_last_combine = []
#         for i in range(len(data_test)//2):
#             data_last = data_test[i:i+1+size_of_gap]

#             data_last[size_of_gap] = model.predict(scaler.transform(np.array(data_last[:size_of_gap]).reshape(1,-1)))

#             results_last_combine.append(data_last[size_of_gap])
            
#         #################################################################
#         print('\n', 'result of combine only (last):')                       #
#         calculate_metrics_for_combine(results_last_combine,'results_combine (LAST)')#
#         print('\n')                                                     #
#         # #################################################################
        
#         df = pd.read_csv('Impute_misvalues_hungyen.csv')

#         plt.figure(figsize=(10, 6))
#         plt.plot(df['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap], label='Water Level')
#         plt.plot(results_last_combine, label='Predicted Value', linestyle='-')
#         plt.title(f'Missing {location}, epochs=200, batch_size=16, patience=30, validation_split=0.2')
#         plt.xlabel('Index')
#         plt.ylabel('Value')
#         plt.grid(True)
#         plt.legend()
#         plt.tight_layout()
#         folder_path = 'P:\SU24\Imputation\HY_git\\128,256,512,128,128'
#         filename = f'128,256,512,128,128, Missing {location}, epochs=200, batch_size=16, patience=30, validation_split=0.2.png'
#         full_path = os.path.join(folder_path, filename)
#         plt.savefig(full_path)
#         plt.show()
        
#     else:
#         Da = data[nan_index + size_of_gap:][::-1]

#         MDa = transform_to_multivariate(Da, size_of_gap)

#         df_MDa = to_df(MDa)

#         X_MDa_train = np.array(df_MDa.iloc[:, :-1])
#         X_MDa_train = scaler.fit_transform(X_MDa_train)
        
#         y_MDa_train = np.array(df_MDa.iloc[:, -1])

#         model_MDa = model_combine(X_MDa_train)
#         model_MDa.fit(X_MDa_train, y_MDa_train, epochs=200, batch_size=16, callbacks=[early_stopping], validation_split=0.2)
        
#         data_test_after = df.values.tolist()[nan_index:nan_index + 2 * size_of_gap ][::-1]
#         data_test_after = np.concatenate(data_test_after).ravel()   

#         value_lst_after = []
#         for j in range(len(data_test_after)//2):
#             data_after = data_test_after[j:j+1+size_of_gap]
            
#             data_after[size_of_gap] = model_MDa.predict(scaler.transform(np.array(data_after[:size_of_gap]).reshape(1,-1)))
            
#             value_lst_after.append(data_after[size_of_gap])

#         Db = data[:nan_index]

#         MDb = transform_to_multivariate(Db, size_of_gap)  

#         df_MDb = to_df(MDb)

#         X_MDb_train = np.array(df_MDb.iloc[:, :-1])
#         X_MDb_train = scaler.fit_transform(X_MDb_train)
        
#         y_MDb_train = np.array(df_MDb.iloc[:, -1])

#         model_MDb = model_combine(X_MDb_train)
#         model_MDb.fit(X_MDb_train, y_MDb_train, epochs=200, batch_size=16, callbacks=[early_stopping], validation_split=0.2)

#         data_test_before = df.values.tolist()[nan_index - size_of_gap : nan_index + size_of_gap]
#         data_test_before = np.concatenate(data_test_before).ravel()   

#         value_lst_before = []
#         for i in range(len(data_test_before)//2):
#             data_before = data_test_before[i:i+1+size_of_gap]
            
#             data_before[size_of_gap] = model_MDb.predict(scaler.transform(np.array(data_before[:size_of_gap]).reshape(1,-1)))

#             value_lst_before.append(data_before[size_of_gap])

#         #############################################################################
#         print('\n', 'result of combine only: ')                                         #
#         results_combine =  [(x + y)/2 for x,y in zip(value_lst_before, value_lst_after)]#
#         calculate_metrics_for_combine(results_combine,'results_combine (MIDDLE)')               #
#         print('\n')                                                                 #
#         #############################################################################
        
#         df = pd.read_csv('Impute_misvalues_hungyen.csv')

#         plt.figure(figsize=(10, 6))
#         plt.plot(df['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap], label='Water Level')
#         plt.plot(results_combine, label='Predicted Value', linestyle='-')
#         plt.title(f'Missing {location}, epochs=200, batch_size=16, patience=30, validation_split=0.2')
#         plt.xlabel('Index')
#         plt.ylabel('Value')
#         plt.grid(True)
#         plt.legend()
#         plt.tight_layout()   
#         folder_path = 'P:\SU24\Imputation\HY_git\\128,256,512,128,128'
#         filename = f'128,256,512,128,128, Missing {location}, epochs=200, batch_size=16, patience=30, validation_split=0.2.png'
#         full_path = os.path.join(folder_path, filename)
#         plt.savefig(full_path)
#         plt.show()
    
    
# current_time = datetime.now()

# print('\n')
# print('\nMean of Similarity combine: ',np.mean(sim_lst_combine))
# print('\nMean of Mean Absoulute Error combine :',np.mean(mae_lst_combine))
# print('\nMean of Root Mean Squared Error combine: ',np.mean(rmse_lst_combine))
# print('\nMean of Fraction of Standard Deviation combine: ',np.mean(fsd_lst_combine)) 
# print('\nMean of R-score combine: ', np.mean(r_lst_combine))
# print('\nMean of the Nash Sutcliffe efficiency (NSE): ', np.mean(nse_lst_combine))
# print('\n')

### epochs=200, batch_size=8, callbacks (patience=30), validation_split=0.2

In [ ]:
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# # tf.debugging.set_log_device_placement(True)

# early_stopping = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)


# for location in range(0,12):
    
#     filename = f'waterlevel_missing_{location}.csv'
#     df = pd.read_csv(filename)
    
#     #### Check size of missing value
#     size_of_gap = df['Waterlevel'].isna().sum()

#     data = df['Waterlevel'].values.tolist()

#     nan_index = None
#     for j, value in enumerate(data):
#         if value != value:  # Check if the value is NaN
#             nan_index = j
#             break
        
#     df_check_before = data[:(3*size_of_gap)+1]
#     df_check_after = data[::-1][:3*size_of_gap+1]
#     df_miss = data[nan_index:nan_index + size_of_gap]

#     last_data = data[:nan_index]
#     first_data = data[nan_index+size_of_gap:][::-1]

#     # check if missing values is in the first 3 x T data original
#     if all(value in df_check_before for value in df_miss):
#         print('\nAll values in df_miss is in the first !!!')

#         first_value_combine = transform_to_multivariate(first_data, size_of_gap)
                
#         df_list = to_df(first_value_combine)
        

#         X = np.array(df_list.iloc[:, :-1])
#         X = scaler.fit_transform(X)
        
#         y = np.array(df_list.iloc[:, -1])
        
#         model = model_combine(X)
#         model.fit(X, y, epochs=200, batch_size=8, callbacks=[early_stopping], validation_split=0.2)

#         data_test = df.values.tolist()[nan_index : nan_index + 2 * size_of_gap][::-1]
#         data_test = np.concatenate(data_test).ravel()   

#         results_first_combine = []
#         for i in range(len(data_test)//2):
#             data_first = data_test[i:i+1+size_of_gap]

#             data_first[size_of_gap] = model.predict(scaler.transform(np.array(data_first[:size_of_gap]).reshape(1,-1)))

#             results_first_combine.append(data_first[size_of_gap])
            
#         ###################################################################
#         print('\n', 'result of combine only (first):')                        #  
#         calculate_metrics_for_combine(results_first_combine,'results_combine (FIRST)')#
#         print('\n')                                                       #
#         ###################################################################
        
#         df = pd.read_csv('Impute_misvalues_hungyen.csv')

#         plt.figure(figsize=(10, 6))
#         plt.plot(df['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap], label='Water Level')
#         plt.plot(results_first_combine, label='Predicted Value', linestyle='-')
#         plt.title(f'Missing {location}, epochs=200, batch_size=8, patience=30, validation_split=0.2')
#         plt.xlabel('Index')
#         plt.ylabel('Value')
#         plt.grid(True)
#         plt.legend()
#         plt.tight_layout()
#         plt.savefig(f'Missing {location}, epochs=200, batch_size=8, patience=30, validation_split=0.2.png')
#         plt.show()
        
#     elif all(value in df_check_after for value in df_miss):
#         print('\nAll values in df_miss is in the last !!!')
        
#         last_value_combine = transform_to_multivariate(last_data, size_of_gap)
                
#         df_list = to_df(last_value_combine)

#         X = np.array(df_list.iloc[:, :-1])
#         X = scaler.fit_transform(X)
        
#         y = np.array(df_list.iloc[:, -1])
        
#         model = model_combine(X)
#         model.fit(X, y, epochs=200, batch_size=8, callbacks=[early_stopping], validation_split=0.2)

#         data_test = df.values.tolist()[nan_index - size_of_gap : nan_index + size_of_gap]
#         data_test = np.concatenate(data_test).ravel()   

#         results_last_combine = []
#         for i in range(len(data_test)//2):
#             data_last = data_test[i:i+1+size_of_gap]

#             data_last[size_of_gap] = model.predict(scaler.transform(np.array(data_last[:size_of_gap]).reshape(1,-1)))

#             results_last_combine.append(data_last[size_of_gap])
            
#         #################################################################
#         print('\n', 'result of combine only (last):')                       #
#         calculate_metrics_for_combine(results_last_combine,'results_combine (LAST)')#
#         print('\n')                                                     #
#         # #################################################################
        
#         df = pd.read_csv('Impute_misvalues_hungyen.csv')

#         plt.figure(figsize=(10, 6))
#         plt.plot(df['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap], label='Water Level')
#         plt.plot(results_last_combine, label='Predicted Value', linestyle='-')
#         plt.title(f'Missing {location}, epochs=200, batch_size=8, patience=30, validation_split=0.2')
#         plt.xlabel('Index')
#         plt.ylabel('Value')
#         plt.grid(True)
#         plt.legend()
#         plt.tight_layout()
#         plt.savefig(f'Missing {location}, epochs=200, batch_size=8, patience=30, validation_split=0.2.png')
#         plt.show()
        
#     else:
#         Da = data[nan_index + size_of_gap:][::-1]

#         MDa = transform_to_multivariate(Da, size_of_gap)

#         df_MDa = to_df(MDa)

#         X_MDa_train = np.array(df_MDa.iloc[:, :-1])
#         X_MDa_train = scaler.fit_transform(X_MDa_train)
        
#         y_MDa_train = np.array(df_MDa.iloc[:, -1])

#         model_MDa = model_combine(X_MDa_train)
#         model_MDa.fit(X_MDa_train, y_MDa_train, epochs=200, batch_size=8, callbacks=[early_stopping], validation_split=0.2)
        
#         data_test_after = df.values.tolist()[nan_index:nan_index + 2 * size_of_gap ][::-1]
#         data_test_after = np.concatenate(data_test_after).ravel()   

#         value_lst_after = []
#         for j in range(len(data_test_after)//2):
#             data_after = data_test_after[j:j+1+size_of_gap]
            
#             data_after[size_of_gap] = model_MDa.predict(scaler.transform(np.array(data_after[:size_of_gap]).reshape(1,-1)))
            
#             value_lst_after.append(data_after[size_of_gap])

#         Db = data[:nan_index]

#         MDb = transform_to_multivariate(Db, size_of_gap)  

#         df_MDb = to_df(MDb)

#         X_MDb_train = np.array(df_MDb.iloc[:, :-1])
#         X_MDb_train = scaler.fit_transform(X_MDb_train)
        
#         y_MDb_train = np.array(df_MDb.iloc[:, -1])

#         model_MDb = model_combine(X_MDb_train)
#         model_MDb.fit(X_MDb_train, y_MDb_train, epochs=200, batch_size=8, callbacks=[early_stopping], validation_split=0.2)

#         data_test_before = df.values.tolist()[nan_index - size_of_gap : nan_index + size_of_gap]
#         data_test_before = np.concatenate(data_test_before).ravel()   

#         value_lst_before = []
#         for i in range(len(data_test_before)//2):
#             data_before = data_test_before[i:i+1+size_of_gap]
            
#             data_before[size_of_gap] = model_MDb.predict(scaler.transform(np.array(data_before[:size_of_gap]).reshape(1,-1)))

#             value_lst_before.append(data_before[size_of_gap])

#         #############################################################################
#         print('\n', 'result of combine only: ')                                         #
#         results_combine =  [(x + y)/2 for x,y in zip(value_lst_before, value_lst_after)]#
#         calculate_metrics_for_combine(results_combine,'results_combine (MIDDLE)')               #
#         print('\n')                                                                 #
#         #############################################################################
        
#         df = pd.read_csv('Impute_misvalues_hungyen.csv')

#         plt.figure(figsize=(10, 6))
#         plt.plot(df['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap], label='Water Level')
#         plt.plot(results_combine, label='Predicted Value', linestyle='-')
#         plt.title(f'Missing {location}, epochs=200, batch_size=8, patience=30, validation_split=0.2')
#         plt.xlabel('Index')
#         plt.ylabel('Value')
#         plt.grid(True)
#         plt.legend()
#         plt.tight_layout()   
#         plt.savefig(f'Missing {location}, epochs=200, batch_size=8, patience=30, validation_split=0.2.png')
#         plt.show()
    
    
# current_time = datetime.now()

# print('\n')
# print('\nMean of Similarity combine: ',np.mean(sim_lst_combine))
# print('\nMean of Mean Absoulute Error combine :',np.mean(mae_lst_combine))
# print('\nMean of Root Mean Squared Error combine: ',np.mean(rmse_lst_combine))
# print('\nMean of Fraction of Standard Deviation combine: ',np.mean(fsd_lst_combine)) 
# print('\nMean of R-score combine: ', np.mean(r_lst_combine))
# print('\nMean of the Nash Sutcliffe efficiency (NSE): ', np.mean(nse_lst_combine))
# print('\n')

### epochs=200, batch_size=32, callbacks (patience=30), validation_split=0.2

In [ ]:
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# # tf.debugging.set_log_device_placement(True)

# early_stopping = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)


# for location in range(0,12):
    
#     filename = f'waterlevel_missing_{location}.csv'
#     df = pd.read_csv(filename)
    
#     #### Check size of missing value
#     size_of_gap = df['Waterlevel'].isna().sum()

#     data = df['Waterlevel'].values.tolist()

#     nan_index = None
#     for j, value in enumerate(data):
#         if value != value:  # Check if the value is NaN
#             nan_index = j
#             break
        
#     df_check_before = data[:(3*size_of_gap)+1]
#     df_check_after = data[::-1][:3*size_of_gap+1]
#     df_miss = data[nan_index:nan_index + size_of_gap]

#     last_data = data[:nan_index]
#     first_data = data[nan_index+size_of_gap:][::-1]

#     # check if missing values is in the first 3 x T data original
#     if all(value in df_check_before for value in df_miss):
#         print('\nAll values in df_miss is in the first !!!')

#         first_value_combine = transform_to_multivariate(first_data, size_of_gap)
                
#         df_list = to_df(first_value_combine)
        

#         X = np.array(df_list.iloc[:, :-1])
#         X = scaler.fit_transform(X)
        
#         y = np.array(df_list.iloc[:, -1])
        
#         model = model_combine(X)
#         model.fit(X, y, epochs=200, batch_size=32, callbacks=[early_stopping], validation_split=0.2)

#         data_test = df.values.tolist()[nan_index : nan_index + 2 * size_of_gap][::-1]
#         data_test = np.concatenate(data_test).ravel()   

#         results_first_combine = []
#         for i in range(len(data_test)//2):
#             data_first = data_test[i:i+1+size_of_gap]

#             data_first[size_of_gap] = model.predict(scaler.transform(np.array(data_first[:size_of_gap]).reshape(1,-1)))

#             results_first_combine.append(data_first[size_of_gap])
            
#         ###################################################################
#         print('\n', 'result of combine only (first):')                        #  
#         calculate_metrics_for_combine(results_first_combine,'results_combine (FIRST)')#
#         print('\n')                                                       #
#         ###################################################################
        
#         df = pd.read_csv('Impute_misvalues_hungyen.csv')

#         plt.figure(figsize=(10, 6))
#         plt.plot(df['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap], label='Water Level')
#         plt.plot(results_first_combine, label='Predicted Value', linestyle='-')
#         plt.title(f'Water Level vs Predicted Value, mising {location}, epochs=200, batch_size=32, patience=30, validation_split=0.2')
#         plt.xlabel('Index')
#         plt.ylabel('Value')
#         plt.grid(True)
#         plt.legend()
#         plt.tight_layout()
#         plt.savefig(f'Missing {location}, epochs=200, batch_size=32, patience=30, validation_split=0.2.png')
#         plt.show()
        
#     elif all(value in df_check_after for value in df_miss):
#         print('\nAll values in df_miss is in the last !!!')
        
#         last_value_combine = transform_to_multivariate(last_data, size_of_gap)
                
#         df_list = to_df(last_value_combine)

#         X = np.array(df_list.iloc[:, :-1])
#         X = scaler.fit_transform(X)
        
#         y = np.array(df_list.iloc[:, -1])
        
#         model = model_combine(X)
#         model.fit(X, y, epochs=200, batch_size=32, callbacks=[early_stopping], validation_split=0.2)

#         data_test = df.values.tolist()[nan_index - size_of_gap : nan_index + size_of_gap]
#         data_test = np.concatenate(data_test).ravel()   

#         results_last_combine = []
#         for i in range(len(data_test)//2):
#             data_last = data_test[i:i+1+size_of_gap]

#             data_last[size_of_gap] = model.predict(scaler.transform(np.array(data_last[:size_of_gap]).reshape(1,-1)))

#             results_last_combine.append(data_last[size_of_gap])
            
#         #################################################################
#         print('\n', 'result of combine only (last):')                       #
#         calculate_metrics_for_combine(results_last_combine,'results_combine (LAST)')#
#         print('\n')                                                     #
#         # #################################################################
        
#         df = pd.read_csv('Impute_misvalues_hungyen.csv')

#         plt.figure(figsize=(10, 6))
#         plt.plot(df['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap], label='Water Level')
#         plt.plot(results_last_combine, label='Predicted Value', linestyle='-')
#         plt.title(f'Water Level vs Predicted Value, mising {location}, epochs=200, batch_size=32, patience=30, validation_split=0.2')
#         plt.xlabel('Index')
#         plt.ylabel('Value')
#         plt.grid(True)
#         plt.legend()
#         plt.tight_layout()
#         plt.savefig(f'Missing {location}, epochs=200, batch_size=32, patience=30, validation_split=0.2.png')
#         plt.show()
        
#     else:
#         Da = data[nan_index + size_of_gap:][::-1]

#         MDa = transform_to_multivariate(Da, size_of_gap)

#         df_MDa = to_df(MDa)

#         X_MDa_train = np.array(df_MDa.iloc[:, :-1])
#         X_MDa_train = scaler.fit_transform(X_MDa_train)
        
#         y_MDa_train = np.array(df_MDa.iloc[:, -1])

#         model_MDa = model_combine(X_MDa_train)
#         model_MDa.fit(X_MDa_train, y_MDa_train, epochs=200, batch_size=32, callbacks=[early_stopping], validation_split=0.2)
        
#         data_test_after = df.values.tolist()[nan_index:nan_index + 2 * size_of_gap ][::-1]
#         data_test_after = np.concatenate(data_test_after).ravel()   

#         value_lst_after = []
#         for j in range(len(data_test_after)//2):
#             data_after = data_test_after[j:j+1+size_of_gap]
            
#             data_after[size_of_gap] = model_MDa.predict(scaler.transform(np.array(data_after[:size_of_gap]).reshape(1,-1)))
            
#             value_lst_after.append(data_after[size_of_gap])

#         Db = data[:nan_index]

#         MDb = transform_to_multivariate(Db, size_of_gap)  

#         df_MDb = to_df(MDb)

#         X_MDb_train = np.array(df_MDb.iloc[:, :-1])
#         X_MDb_train = scaler.fit_transform(X_MDb_train)
        
#         y_MDb_train = np.array(df_MDb.iloc[:, -1])

#         model_MDb = model_combine(X_MDb_train)
#         model_MDb.fit(X_MDb_train, y_MDb_train, epochs=200, batch_size=32, callbacks=[early_stopping], validation_split=0.2)

#         data_test_before = df.values.tolist()[nan_index - size_of_gap : nan_index + size_of_gap]
#         data_test_before = np.concatenate(data_test_before).ravel()   

#         value_lst_before = []
#         for i in range(len(data_test_before)//2):
#             data_before = data_test_before[i:i+1+size_of_gap]
            
#             data_before[size_of_gap] = model_MDb.predict(scaler.transform(np.array(data_before[:size_of_gap]).reshape(1,-1)))

#             value_lst_before.append(data_before[size_of_gap])

#         #############################################################################
#         print('\n', 'result of combine only: ')                                         #
#         results_combine =  [(x + y)/2 for x,y in zip(value_lst_before, value_lst_after)]#
#         calculate_metrics_for_combine(results_combine,'results_combine (MIDDLE)')               #
#         print('\n')                                                                 #
#         #############################################################################
        
#         df = pd.read_csv('Impute_misvalues_hungyen.csv')

#         plt.figure(figsize=(10, 6))
#         plt.plot(df['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap], label='Water Level')
#         plt.plot(results_combine, label='Predicted Value', linestyle='-')
#         plt.title(f'Water Level vs Predicted Value, mising {location}, epochs=200, batch_size=32, patience=30, validation_split=0.2')
#         plt.xlabel('Index')
#         plt.ylabel('Value')
#         plt.grid(True)
#         plt.legend()
#         plt.tight_layout()
#         plt.show()    
#         plt.savefig(f'Missing {location}, epochs=200, batch_size=32, patience=30, validation_split=0.2.png')
    
    
# current_time = datetime.now()

# print('\n')
# print('\nMean of Similarity combine: ',np.mean(sim_lst_combine))
# print('\nMean of Mean Absoulute Error combine :',np.mean(mae_lst_combine))
# print('\nMean of Root Mean Squared Error combine: ',np.mean(rmse_lst_combine))
# print('\nMean of Fraction of Standard Deviation combine: ',np.mean(fsd_lst_combine)) 
# print('\nMean of R-score combine: ', np.mean(r_lst_combine))
# print('\nMean of the Nash Sutcliffe efficiency (NSE): ', np.mean(nse_lst_combine))
# print('\n')

### epochs=200, batch_size=8, callbacks (patience=50), validation_split=0.2

In [ ]:
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# # tf.debugging.set_log_device_placement(True)

# early_stopping = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)


# for location in range(0,12):
    
#     filename = f'waterlevel_missing_{location}.csv'
#     df = pd.read_csv(filename)
    
#     #### Check size of missing value
#     size_of_gap = df['Waterlevel'].isna().sum()

#     data = df['Waterlevel'].values.tolist()

#     nan_index = None
#     for j, value in enumerate(data):
#         if value != value:  # Check if the value is NaN
#             nan_index = j
#             break
        
#     df_check_before = data[:(3*size_of_gap)+1]
#     df_check_after = data[::-1][:3*size_of_gap+1]
#     df_miss = data[nan_index:nan_index + size_of_gap]

#     last_data = data[:nan_index]
#     first_data = data[nan_index+size_of_gap:][::-1]

#     # check if missing values is in the first 3 x T data original
#     if all(value in df_check_before for value in df_miss):
#         print('\nAll values in df_miss is in the first !!!')

#         first_value_combine = transform_to_multivariate(first_data, size_of_gap)
                
#         df_list = to_df(first_value_combine)
        

#         X = np.array(df_list.iloc[:, :-1])
#         X = scaler.fit_transform(X)
        
#         y = np.array(df_list.iloc[:, -1])
        
#         model = model_combine(X)
#         model.fit(X, y, epochs=200, batch_size=8, callbacks=[early_stopping], validation_split=0.2)

#         data_test = df.values.tolist()[nan_index : nan_index + 2 * size_of_gap][::-1]
#         data_test = np.concatenate(data_test).ravel()   

#         results_first_combine = []
#         for i in range(len(data_test)//2):
#             data_first = data_test[i:i+1+size_of_gap]

#             data_first[size_of_gap] = model.predict(scaler.transform(np.array(data_first[:size_of_gap]).reshape(1,-1)))

#             results_first_combine.append(data_first[size_of_gap])
            
#         ###################################################################
#         print('\n', 'result of combine only (first):')                        #  
#         calculate_metrics_for_combine(results_first_combine,'results_combine (FIRST)')#
#         print('\n')                                                       #
#         ###################################################################
        
#         df = pd.read_csv('Impute_misvalues_hungyen.csv')

#         plt.figure(figsize=(10, 6))
#         plt.plot(df['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap], label='Water Level')
#         plt.plot(results_first_combine, label='Predicted Value', linestyle='-')
#         plt.title(f'Water Level vs Predicted Value, mising {location}, epochs=200, batch_size=8, patience=50, validation_split=0.2')
#         plt.xlabel('Index')
#         plt.ylabel('Value')
#         plt.grid(True)
#         plt.legend()
#         plt.tight_layout()
#         plt.savefig(f'Missing {location}, epochs=200, batch_size=8, patience=50, validation_split=0.2.png')
#         plt.show()
        
#     elif all(value in df_check_after for value in df_miss):
#         print('\nAll values in df_miss is in the last !!!')
        
#         last_value_combine = transform_to_multivariate(last_data, size_of_gap)
                
#         df_list = to_df(last_value_combine)

#         X = np.array(df_list.iloc[:, :-1])
#         X = scaler.fit_transform(X)
        
#         y = np.array(df_list.iloc[:, -1])
        
#         model = model_combine(X)
#         model.fit(X, y, epochs=200, batch_size=8, callbacks=[early_stopping], validation_split=0.2)

#         data_test = df.values.tolist()[nan_index - size_of_gap : nan_index + size_of_gap]
#         data_test = np.concatenate(data_test).ravel()   

#         results_last_combine = []
#         for i in range(len(data_test)//2):
#             data_last = data_test[i:i+1+size_of_gap]

#             data_last[size_of_gap] = model.predict(scaler.transform(np.array(data_last[:size_of_gap]).reshape(1,-1)))

#             results_last_combine.append(data_last[size_of_gap])
            
#         #################################################################
#         print('\n', 'result of combine only (last):')                       #
#         calculate_metrics_for_combine(results_last_combine,'results_combine (LAST)')#
#         print('\n')                                                     #
#         # #################################################################
        
#         df = pd.read_csv('Impute_misvalues_hungyen.csv')

#         plt.figure(figsize=(10, 6))
#         plt.plot(df['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap], label='Water Level')
#         plt.plot(results_last_combine, label='Predicted Value', linestyle='-')
#         plt.title(f'Water Level vs Predicted Value, mising {location}, epochs=200, batch_size=8, patience=50, validation_split=0.2')
#         plt.xlabel('Index')
#         plt.ylabel('Value')
#         plt.grid(True)
#         plt.legend()
#         plt.tight_layout()
#         plt.savefig(f'Missing {location}, epochs=200, batch_size=8, patience=50, validation_split=0.2.png')
#         plt.show()
        
#     else:
#         Da = data[nan_index + size_of_gap:][::-1]

#         MDa = transform_to_multivariate(Da, size_of_gap)

#         df_MDa = to_df(MDa)

#         X_MDa_train = np.array(df_MDa.iloc[:, :-1])
#         X_MDa_train = scaler.fit_transform(X_MDa_train)
        
#         y_MDa_train = np.array(df_MDa.iloc[:, -1])

#         model_MDa = model_combine(X_MDa_train)
#         model_MDa.fit(X_MDa_train, y_MDa_train, epochs=200, batch_size=8, callbacks=[early_stopping], validation_split=0.2)
        
#         data_test_after = df.values.tolist()[nan_index:nan_index + 2 * size_of_gap ][::-1]
#         data_test_after = np.concatenate(data_test_after).ravel()   

#         value_lst_after = []
#         for j in range(len(data_test_after)//2):
#             data_after = data_test_after[j:j+1+size_of_gap]
            
#             data_after[size_of_gap] = model_MDa.predict(scaler.transform(np.array(data_after[:size_of_gap]).reshape(1,-1)))
            
#             value_lst_after.append(data_after[size_of_gap])

#         Db = data[:nan_index]

#         MDb = transform_to_multivariate(Db, size_of_gap)  

#         df_MDb = to_df(MDb)

#         X_MDb_train = np.array(df_MDb.iloc[:, :-1])
#         X_MDb_train = scaler.fit_transform(X_MDb_train)
        
#         y_MDb_train = np.array(df_MDb.iloc[:, -1])

#         model_MDb = model_combine(X_MDb_train)
#         model_MDb.fit(X_MDb_train, y_MDb_train, epochs=200, batch_size=8, callbacks=[early_stopping], validation_split=0.2)

#         data_test_before = df.values.tolist()[nan_index - size_of_gap : nan_index + size_of_gap]
#         data_test_before = np.concatenate(data_test_before).ravel()   

#         value_lst_before = []
#         for i in range(len(data_test_before)//2):
#             data_before = data_test_before[i:i+1+size_of_gap]
            
#             data_before[size_of_gap] = model_MDb.predict(scaler.transform(np.array(data_before[:size_of_gap]).reshape(1,-1)))

#             value_lst_before.append(data_before[size_of_gap])

#         #############################################################################
#         print('\n', 'result of combine only: ')                                         #
#         results_combine =  [(x + y)/2 for x,y in zip(value_lst_before, value_lst_after)]#
#         calculate_metrics_for_combine(results_combine,'results_combine (MIDDLE)')               #
#         print('\n')                                                                 #
#         #############################################################################
        
#         df = pd.read_csv('Impute_misvalues_hungyen.csv')

#         plt.figure(figsize=(10, 6))
#         plt.plot(df['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap], label='Water Level')
#         plt.plot(results_combine, label='Predicted Value', linestyle='-')
#         plt.title(f'Water Level vs Predicted Value, mising {location}, epochs=200, batch_size=8, patience=50, validation_split=0.2')
#         plt.xlabel('Index')
#         plt.ylabel('Value')
#         plt.grid(True)
#         plt.legend()
#         plt.tight_layout()
#         plt.show()    
#         plt.savefig(f'Missing {location}, epochs=200, batch_size=8, patience=50, validation_split=0.2.png')
    
    
# current_time = datetime.now()

# print('\n')
# print('\nMean of Similarity combine: ',np.mean(sim_lst_combine))
# print('\nMean of Mean Absoulute Error combine :',np.mean(mae_lst_combine))
# print('\nMean of Root Mean Squared Error combine: ',np.mean(rmse_lst_combine))
# print('\nMean of Fraction of Standard Deviation combine: ',np.mean(fsd_lst_combine)) 
# print('\nMean of R-score combine: ', np.mean(r_lst_combine))
# print('\nMean of the Nash Sutcliffe efficiency (NSE): ', np.mean(nse_lst_combine))
# print('\n')

### epochs=200, batch_size=16, callbacks (patience=50), validation_split=0.2

In [ ]:
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# # tf.debugging.set_log_device_placement(True)

# early_stopping = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)


# for location in range(0,12):
    
#     filename = f'P:\SU24\Imputation\HY_git\waterlevel_missing\waterlevel_missing_{location}.csv'
#     df = pd.read_csv(filename)
    
#     #### Check size of missing value
#     size_of_gap = df['Waterlevel'].isna().sum()

#     data = df['Waterlevel'].values.tolist()

#     nan_index = None
#     for j, value in enumerate(data):
#         if value != value:  # Check if the value is NaN
#             nan_index = j
#             break
        
#     df_check_before = data[:(3*size_of_gap)+1]
#     df_check_after = data[::-1][:3*size_of_gap+1]
#     df_miss = data[nan_index:nan_index + size_of_gap]

#     last_data = data[:nan_index]
#     first_data = data[nan_index+size_of_gap:][::-1]

#     # check if missing values is in the first 3 x T data original
#     if all(value in df_check_before for value in df_miss):
#         print('\nAll values in df_miss is in the first !!!')

#         first_value_combine = transform_to_multivariate(first_data, size_of_gap)
                
#         df_list = to_df(first_value_combine)
        

#         X = np.array(df_list.iloc[:, :-1])
#         X = scaler.fit_transform(X)
        
#         y = np.array(df_list.iloc[:, -1])
        
#         model = model_combine(X)
#         model.fit(X, y, epochs=200, batch_size=16, callbacks=[early_stopping], validation_split=0.2)

#         data_test = df.values.tolist()[nan_index : nan_index + 2 * size_of_gap][::-1]
#         data_test = np.concatenate(data_test).ravel()   

#         results_first_combine = []
#         for i in range(len(data_test)//2):
#             data_first = data_test[i:i+1+size_of_gap]

#             data_first[size_of_gap] = model.predict(scaler.transform(np.array(data_first[:size_of_gap]).reshape(1,-1)))

#             results_first_combine.append(data_first[size_of_gap])
            
#         ###################################################################
#         print('\n', 'result of combine only (first):')                        #  
#         calculate_metrics_for_combine(results_first_combine,'results_combine (FIRST)')#
#         print('\n')                                                       #
#         ###################################################################
        
#         df = pd.read_csv('Impute_misvalues_hungyen.csv')

#         plt.figure(figsize=(10, 6))
#         plt.plot(df['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap], label='Water Level')
#         plt.plot(results_first_combine, label='Predicted Value', linestyle='-')
#         plt.title(f'Mising {location}, epochs=200, batch_size=16, patience=50, validation_split=0.2')
#         plt.xlabel('Index')
#         plt.ylabel('Value')
#         plt.grid(True)
#         plt.legend()
#         plt.tight_layout()
#         folder_path = 'P:\SU24\Imputation\HY_git\\128,256,512,64,64'
#         filename = f'128,256,512,64,64, Missing {location}, epochs=200, batch_size=16, patience=50, validation_split=0.2.png'
#         full_path = os.path.join(folder_path, filename)
#         plt.savefig(full_path)
#         plt.show()
        
#     elif all(value in df_check_after for value in df_miss):
#         print('\nAll values in df_miss is in the last !!!')
        
#         last_value_combine = transform_to_multivariate(last_data, size_of_gap)
                
#         df_list = to_df(last_value_combine)

#         X = np.array(df_list.iloc[:, :-1])
#         X = scaler.fit_transform(X)
        
#         y = np.array(df_list.iloc[:, -1])
        
#         model = model_combine(X)
#         model.fit(X, y, epochs=200, batch_size=16, callbacks=[early_stopping], validation_split=0.2)

#         data_test = df.values.tolist()[nan_index - size_of_gap : nan_index + size_of_gap]
#         data_test = np.concatenate(data_test).ravel()   

#         results_last_combine = []
#         for i in range(len(data_test)//2):
#             data_last = data_test[i:i+1+size_of_gap]

#             data_last[size_of_gap] = model.predict(scaler.transform(np.array(data_last[:size_of_gap]).reshape(1,-1)))

#             results_last_combine.append(data_last[size_of_gap])
            
#         #################################################################
#         print('\n', 'result of combine only (last):')                       #
#         calculate_metrics_for_combine(results_last_combine,'results_combine (LAST)')#
#         print('\n')                                                     #
#         # #################################################################
        
#         df = pd.read_csv('Impute_misvalues_hungyen.csv')

#         plt.figure(figsize=(10, 6))
#         plt.plot(df['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap], label='Water Level')
#         plt.plot(results_last_combine, label='Predicted Value', linestyle='-')
#         plt.title(f'Mising {location}, epochs=200, batch_size=16, patience=50, validation_split=0.2')
#         plt.xlabel('Index')
#         plt.ylabel('Value')
#         plt.grid(True)
#         plt.legend()
#         plt.tight_layout()
#         folder_path = 'P:\SU24\Imputation\HY_git\\128,256,512,64,64'
#         filename = f'128,256,512,64,64, Missing {location}, epochs=200, batch_size=16, patience=50, validation_split=0.2.png'
#         full_path = os.path.join(folder_path, filename)
#         plt.savefig(full_path)
#         plt.show()
        
#     else:
#         Da = data[nan_index + size_of_gap:][::-1]

#         MDa = transform_to_multivariate(Da, size_of_gap)

#         df_MDa = to_df(MDa)

#         X_MDa_train = np.array(df_MDa.iloc[:, :-1])
#         X_MDa_train = scaler.fit_transform(X_MDa_train)
        
#         y_MDa_train = np.array(df_MDa.iloc[:, -1])

#         model_MDa = model_combine(X_MDa_train)
#         model_MDa.fit(X_MDa_train, y_MDa_train, epochs=200, batch_size=16, callbacks=[early_stopping], validation_split=0.2)
        
#         data_test_after = df.values.tolist()[nan_index:nan_index + 2 * size_of_gap ][::-1]
#         data_test_after = np.concatenate(data_test_after).ravel()   

#         value_lst_after = []
#         for j in range(len(data_test_after)//2):
#             data_after = data_test_after[j:j+1+size_of_gap]
            
#             data_after[size_of_gap] = model_MDa.predict(scaler.transform(np.array(data_after[:size_of_gap]).reshape(1,-1)))
            
#             value_lst_after.append(data_after[size_of_gap])

#         Db = data[:nan_index]

#         MDb = transform_to_multivariate(Db, size_of_gap)  

#         df_MDb = to_df(MDb)

#         X_MDb_train = np.array(df_MDb.iloc[:, :-1])
#         X_MDb_train = scaler.fit_transform(X_MDb_train)
        
#         y_MDb_train = np.array(df_MDb.iloc[:, -1])

#         model_MDb = model_combine(X_MDb_train)
#         model_MDb.fit(X_MDb_train, y_MDb_train, epochs=200, batch_size=16, callbacks=[early_stopping], validation_split=0.2)

#         data_test_before = df.values.tolist()[nan_index - size_of_gap : nan_index + size_of_gap]
#         data_test_before = np.concatenate(data_test_before).ravel()   

#         value_lst_before = []
#         for i in range(len(data_test_before)//2):
#             data_before = data_test_before[i:i+1+size_of_gap]
            
#             data_before[size_of_gap] = model_MDb.predict(scaler.transform(np.array(data_before[:size_of_gap]).reshape(1,-1)))

#             value_lst_before.append(data_before[size_of_gap])

#         #############################################################################
#         print('\n', 'result of combine only: ')                                         #
#         results_combine =  [(x + y)/2 for x,y in zip(value_lst_before, value_lst_after)]#
#         calculate_metrics_for_combine(results_combine,'results_combine (MIDDLE)')               #
#         print('\n')                                                                 #
#         #############################################################################
        
#         df = pd.read_csv('Impute_misvalues_hungyen.csv')

#         plt.figure(figsize=(10, 6))
#         plt.plot(df['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap], label='Water Level')
#         plt.plot(results_combine, label='Predicted Value', linestyle='-')
#         plt.title(f'Mising {location}, epochs=200, batch_size=16, patience=50, validation_split=0.2')
#         plt.xlabel('Index')
#         plt.ylabel('Value')
#         plt.grid(True)
#         plt.legend()
#         plt.tight_layout()
#         plt.show()    
#         folder_path = 'P:\SU24\Imputation\HY_git\\128,256,512,64,64'
#         filename = f'128,256,512,64,64, Missing {location}, epochs=200, batch_size=16, patience=50, validation_split=0.2.png'
#         full_path = os.path.join(folder_path, filename)
#         plt.savefig(full_path)
    
    
# current_time = datetime.now()

# print('\n')
# print('\nMean of Similarity combine: ',np.mean(sim_lst_combine))
# print('\nMean of Mean Absoulute Error combine :',np.mean(mae_lst_combine))
# print('\nMean of Root Mean Squared Error combine: ',np.mean(rmse_lst_combine))
# print('\nMean of Fraction of Standard Deviation combine: ',np.mean(fsd_lst_combine)) 
# print('\nMean of R-score combine: ', np.mean(r_lst_combine))
# print('\nMean of the Nash Sutcliffe efficiency (NSE): ', np.mean(nse_lst_combine))
# print('\n')

### epochs=200, batch_size=32, callbacks (patience=50), validation_split=0.2

In [ ]:
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# # tf.debugging.set_log_device_placement(True)

# early_stopping = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)


# for location in range(0,12):
    
#     filename = f'waterlevel_missing_{location}.csv'
#     df = pd.read_csv(filename)
    
#     #### Check size of missing value
#     size_of_gap = df['Waterlevel'].isna().sum()

#     data = df['Waterlevel'].values.tolist()

#     nan_index = None
#     for j, value in enumerate(data):
#         if value != value:  # Check if the value is NaN
#             nan_index = j
#             break
        
#     df_check_before = data[:(3*size_of_gap)+1]
#     df_check_after = data[::-1][:3*size_of_gap+1]
#     df_miss = data[nan_index:nan_index + size_of_gap]

#     last_data = data[:nan_index]
#     first_data = data[nan_index+size_of_gap:][::-1]

#     # check if missing values is in the first 3 x T data original
#     if all(value in df_check_before for value in df_miss):
#         print('\nAll values in df_miss is in the first !!!')

#         first_value_combine = transform_to_multivariate(first_data, size_of_gap)
                
#         df_list = to_df(first_value_combine)
        

#         X = np.array(df_list.iloc[:, :-1])
#         X = scaler.fit_transform(X)
        
#         y = np.array(df_list.iloc[:, -1])
        
#         model = model_combine(X)
#         model.fit(X, y, epochs=200, batch_size=32, callbacks=[early_stopping], validation_split=0.2)

#         data_test = df.values.tolist()[nan_index : nan_index + 2 * size_of_gap][::-1]
#         data_test = np.concatenate(data_test).ravel()   

#         results_first_combine = []
#         for i in range(len(data_test)//2):
#             data_first = data_test[i:i+1+size_of_gap]

#             data_first[size_of_gap] = model.predict(scaler.transform(np.array(data_first[:size_of_gap]).reshape(1,-1)))

#             results_first_combine.append(data_first[size_of_gap])
            
#         ###################################################################
#         print('\n', 'result of combine only (first):')                        #  
#         calculate_metrics_for_combine(results_first_combine,'results_combine (FIRST)')#
#         print('\n')                                                       #
#         ###################################################################
        
#         df = pd.read_csv('Impute_misvalues_hungyen.csv')

#         plt.figure(figsize=(10, 6))
#         plt.plot(df['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap], label='Water Level')
#         plt.plot(results_first_combine, label='Predicted Value', linestyle='-')
#         plt.title(f'Water Level vs Predicted Value, mising {location}, epochs=200, batch_size=32, callbacks (patience=50), validation_split=0.2')
#         plt.xlabel('Index')
#         plt.ylabel('Value')
#         plt.grid(True)
#         plt.legend()
#         plt.tight_layout()
#         plt.savefig(f'Missing {location}, epochs=200, batch_size=32, callbacks (patience=50), validation_split=0.2.png')
#         plt.show()
        
#     elif all(value in df_check_after for value in df_miss):
#         print('\nAll values in df_miss is in the last !!!')
        
#         last_value_combine = transform_to_multivariate(last_data, size_of_gap)
                
#         df_list = to_df(last_value_combine)

#         X = np.array(df_list.iloc[:, :-1])
#         X = scaler.fit_transform(X)
        
#         y = np.array(df_list.iloc[:, -1])
        
#         model = model_combine(X)
#         model.fit(X, y, epochs=200, batch_size=32, callbacks=[early_stopping], validation_split=0.2)

#         data_test = df.values.tolist()[nan_index - size_of_gap : nan_index + size_of_gap]
#         data_test = np.concatenate(data_test).ravel()   

#         results_last_combine = []
#         for i in range(len(data_test)//2):
#             data_last = data_test[i:i+1+size_of_gap]

#             data_last[size_of_gap] = model.predict(scaler.transform(np.array(data_last[:size_of_gap]).reshape(1,-1)))

#             results_last_combine.append(data_last[size_of_gap])
            
#         #################################################################
#         print('\n', 'result of combine only (last):')                       #
#         calculate_metrics_for_combine(results_last_combine,'results_combine (LAST)')#
#         print('\n')                                                     #
#         # #################################################################
        
#         df = pd.read_csv('Impute_misvalues_hungyen.csv')

#         plt.figure(figsize=(10, 6))
#         plt.plot(df['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap], label='Water Level')
#         plt.plot(results_last_combine, label='Predicted Value', linestyle='-')
#         plt.title(f'Water Level vs Predicted Value, mising {location}, epochs=200, batch_size=32, callbacks (patience=50), validation_split=0.2')
#         plt.xlabel('Index')
#         plt.ylabel('Value')
#         plt.grid(True)
#         plt.legend()
#         plt.tight_layout()
#         plt.savefig(f'Missing {location}, epochs=200, batch_size=32, callbacks (patience=50), validation_split=0.2.png')
#         plt.show()
        
#     else:
#         Da = data[nan_index + size_of_gap:][::-1]

#         MDa = transform_to_multivariate(Da, size_of_gap)

#         df_MDa = to_df(MDa)

#         X_MDa_train = np.array(df_MDa.iloc[:, :-1])
#         X_MDa_train = scaler.fit_transform(X_MDa_train)
        
#         y_MDa_train = np.array(df_MDa.iloc[:, -1])

#         model_MDa = model_combine(X_MDa_train)
#         model_MDa.fit(X_MDa_train, y_MDa_train, epochs=200, batch_size=32, callbacks=[early_stopping], validation_split=0.2)
        
#         data_test_after = df.values.tolist()[nan_index:nan_index + 2 * size_of_gap ][::-1]
#         data_test_after = np.concatenate(data_test_after).ravel()   

#         value_lst_after = []
#         for j in range(len(data_test_after)//2):
#             data_after = data_test_after[j:j+1+size_of_gap]
            
#             data_after[size_of_gap] = model_MDa.predict(scaler.transform(np.array(data_after[:size_of_gap]).reshape(1,-1)))
            
#             value_lst_after.append(data_after[size_of_gap])

#         Db = data[:nan_index]

#         MDb = transform_to_multivariate(Db, size_of_gap)  

#         df_MDb = to_df(MDb)

#         X_MDb_train = np.array(df_MDb.iloc[:, :-1])
#         X_MDb_train = scaler.fit_transform(X_MDb_train)
        
#         y_MDb_train = np.array(df_MDb.iloc[:, -1])

#         model_MDb = model_combine(X_MDb_train)
#         model_MDb.fit(X_MDb_train, y_MDb_train, epochs=200, batch_size=32, callbacks=[early_stopping], validation_split=0.2)

#         data_test_before = df.values.tolist()[nan_index - size_of_gap : nan_index + size_of_gap]
#         data_test_before = np.concatenate(data_test_before).ravel()   

#         value_lst_before = []
#         for i in range(len(data_test_before)//2):
#             data_before = data_test_before[i:i+1+size_of_gap]
            
#             data_before[size_of_gap] = model_MDb.predict(scaler.transform(np.array(data_before[:size_of_gap]).reshape(1,-1)))

#             value_lst_before.append(data_before[size_of_gap])

#         #############################################################################
#         print('\n', 'result of combine only: ')                                         #
#         results_combine =  [(x + y)/2 for x,y in zip(value_lst_before, value_lst_after)]#
#         calculate_metrics_for_combine(results_combine,'results_combine (MIDDLE)')               #
#         print('\n')                                                                 #
#         #############################################################################
        
#         df = pd.read_csv('Impute_misvalues_hungyen.csv')

#         plt.figure(figsize=(10, 6))
#         plt.plot(df['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap], label='Water Level')
#         plt.plot(results_combine, label='Predicted Value', linestyle='-')
#         plt.title(f'Water Level vs Predicted Value, mising {location}, epochs=200, batch_size=32, callbacks (patience=50), validation_split=0.2')
#         plt.xlabel('Index')
#         plt.ylabel('Value')
#         plt.grid(True)
#         plt.legend()
#         plt.tight_layout()
#         plt.show()    
#         plt.savefig(f'Missing {location}, epochs=200, batch_size=32, callbacks (patience=50), validation_split=0.2.png')
    
    
# current_time = datetime.now()

# print('\n')
# print('\nMean of Similarity combine: ',np.mean(sim_lst_combine))
# print('\nMean of Mean Absoulute Error combine :',np.mean(mae_lst_combine))
# print('\nMean of Root Mean Squared Error combine: ',np.mean(rmse_lst_combine))
# print('\nMean of Fraction of Standard Deviation combine: ',np.mean(fsd_lst_combine)) 
# print('\nMean of R-score combine: ', np.mean(r_lst_combine))
# print('\nMean of the Nash Sutcliffe efficiency (NSE): ', np.mean(nse_lst_combine))
# print('\n')

### epochs=200, batch_size=64, callbacks (patience=50), validation_split=0.2

In [ ]:
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# # tf.debugging.set_log_device_placement(True)

# early_stopping = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)


# for location in range(0,12):
    
#     filename = f'waterlevel_missing_{location}.csv'
#     df = pd.read_csv(filename)
    
#     #### Check size of missing value
#     size_of_gap = df['Waterlevel'].isna().sum()

#     data = df['Waterlevel'].values.tolist()

#     nan_index = None
#     for j, value in enumerate(data):
#         if value != value:  # Check if the value is NaN
#             nan_index = j
#             break
        
#     df_check_before = data[:(3*size_of_gap)+1]
#     df_check_after = data[::-1][:3*size_of_gap+1]
#     df_miss = data[nan_index:nan_index + size_of_gap]

#     last_data = data[:nan_index]
#     first_data = data[nan_index+size_of_gap:][::-1]

#     # check if missing values is in the first 3 x T data original
#     if all(value in df_check_before for value in df_miss):
#         print('\nAll values in df_miss is in the first !!!')

#         first_value_combine = transform_to_multivariate(first_data, size_of_gap)
                
#         df_list = to_df(first_value_combine)
        

#         X = np.array(df_list.iloc[:, :-1])
#         X = scaler.fit_transform(X)
        
#         y = np.array(df_list.iloc[:, -1])
        
#         model = model_combine(X)
#         model.fit(X, y, epochs=200, batch_size=64, callbacks=[early_stopping], validation_split=0.2)

#         data_test = df.values.tolist()[nan_index : nan_index + 2 * size_of_gap][::-1]
#         data_test = np.concatenate(data_test).ravel()   

#         results_first_combine = []
#         for i in range(len(data_test)//2):
#             data_first = data_test[i:i+1+size_of_gap]

#             data_first[size_of_gap] = model.predict(scaler.transform(np.array(data_first[:size_of_gap]).reshape(1,-1)))

#             results_first_combine.append(data_first[size_of_gap])
            
#         ###################################################################
#         print('\n', 'result of combine only (first):')                        #  
#         calculate_metrics_for_combine(results_first_combine,'results_combine (FIRST)')#
#         print('\n')                                                       #
#         ###################################################################
        
#         df = pd.read_csv('Impute_misvalues_hungyen.csv')

#         plt.figure(figsize=(10, 6))
#         plt.plot(df['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap], label='Water Level')
#         plt.plot(results_first_combine, label='Predicted Value', linestyle='-')
#         plt.title(f'Water Level vs Predicted Value, mising {location}, epochs=200, batch_size=64, callbacks (patience=50), validation_split=0.2')
#         plt.xlabel('Index')
#         plt.ylabel('Value')
#         plt.grid(True)
#         plt.legend()
#         plt.tight_layout()
#         plt.savefig(f'Missing {location}, epochs=200, batch_size=64, callbacks (patience=50), validation_split=0.2.png')
#         plt.show()
        
#     elif all(value in df_check_after for value in df_miss):
#         print('\nAll values in df_miss is in the last !!!')
        
#         last_value_combine = transform_to_multivariate(last_data, size_of_gap)
                
#         df_list = to_df(last_value_combine)

#         X = np.array(df_list.iloc[:, :-1])
#         X = scaler.fit_transform(X)
        
#         y = np.array(df_list.iloc[:, -1])
        
#         model = model_combine(X)
#         model.fit(X, y, epochs=200, batch_size=64, callbacks=[early_stopping], validation_split=0.2)

#         data_test = df.values.tolist()[nan_index - size_of_gap : nan_index + size_of_gap]
#         data_test = np.concatenate(data_test).ravel()   

#         results_last_combine = []
#         for i in range(len(data_test)//2):
#             data_last = data_test[i:i+1+size_of_gap]

#             data_last[size_of_gap] = model.predict(scaler.transform(np.array(data_last[:size_of_gap]).reshape(1,-1)))

#             results_last_combine.append(data_last[size_of_gap])
            
#         #################################################################
#         print('\n', 'result of combine only (last):')                       #
#         calculate_metrics_for_combine(results_last_combine,'results_combine (LAST)')#
#         print('\n')                                                     #
#         # #################################################################
        
#         df = pd.read_csv('Impute_misvalues_hungyen.csv')

#         plt.figure(figsize=(10, 6))
#         plt.plot(df['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap], label='Water Level')
#         plt.plot(results_last_combine, label='Predicted Value', linestyle='-')
#         plt.title(f'Water Level vs Predicted Value, mising {location}, epochs=200, batch_size=64, callbacks (patience=50), validation_split=0.2')
#         plt.xlabel('Index')
#         plt.ylabel('Value')
#         plt.grid(True)
#         plt.legend()
#         plt.tight_layout()
#         plt.savefig(f'Missing {location}, epochs=200, batch_size=64, callbacks (patience=50), validation_split=0.2.png')
#         plt.show()
        
#     else:
#         Da = data[nan_index + size_of_gap:][::-1]

#         MDa = transform_to_multivariate(Da, size_of_gap)

#         df_MDa = to_df(MDa)

#         X_MDa_train = np.array(df_MDa.iloc[:, :-1])
#         X_MDa_train = scaler.fit_transform(X_MDa_train)
        
#         y_MDa_train = np.array(df_MDa.iloc[:, -1])

#         model_MDa = model_combine(X_MDa_train)
#         model_MDa.fit(X_MDa_train, y_MDa_train, epochs=200, batch_size=64, callbacks=[early_stopping], validation_split=0.2)
        
#         data_test_after = df.values.tolist()[nan_index:nan_index + 2 * size_of_gap ][::-1]
#         data_test_after = np.concatenate(data_test_after).ravel()   

#         value_lst_after = []
#         for j in range(len(data_test_after)//2):
#             data_after = data_test_after[j:j+1+size_of_gap]
            
#             data_after[size_of_gap] = model_MDa.predict(scaler.transform(np.array(data_after[:size_of_gap]).reshape(1,-1)))
            
#             value_lst_after.append(data_after[size_of_gap])

#         Db = data[:nan_index]

#         MDb = transform_to_multivariate(Db, size_of_gap)  

#         df_MDb = to_df(MDb)

#         X_MDb_train = np.array(df_MDb.iloc[:, :-1])
#         X_MDb_train = scaler.fit_transform(X_MDb_train)
        
#         y_MDb_train = np.array(df_MDb.iloc[:, -1])

#         model_MDb = model_combine(X_MDb_train)
#         model_MDb.fit(X_MDb_train, y_MDb_train, epochs=200, batch_size=64, callbacks=[early_stopping], validation_split=0.2)

#         data_test_before = df.values.tolist()[nan_index - size_of_gap : nan_index + size_of_gap]
#         data_test_before = np.concatenate(data_test_before).ravel()   

#         value_lst_before = []
#         for i in range(len(data_test_before)//2):
#             data_before = data_test_before[i:i+1+size_of_gap]
            
#             data_before[size_of_gap] = model_MDb.predict(scaler.transform(np.array(data_before[:size_of_gap]).reshape(1,-1)))

#             value_lst_before.append(data_before[size_of_gap])

#         #############################################################################
#         print('\n', 'result of combine only: ')                                         #
#         results_combine =  [(x + y)/2 for x,y in zip(value_lst_before, value_lst_after)]#
#         calculate_metrics_for_combine(results_combine,'results_combine (MIDDLE)')               #
#         print('\n')                                                                 #
#         #############################################################################
        
#         df = pd.read_csv('Impute_misvalues_hungyen.csv')

#         plt.figure(figsize=(10, 6))
#         plt.plot(df['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap], label='Water Level')
#         plt.plot(results_combine, label='Predicted Value', linestyle='-')
#         plt.title(f'Water Level vs Predicted Value, mising {location}, epochs=200, batch_size=64, callbacks (patience=50), validation_split=0.2')
#         plt.xlabel('Index')
#         plt.ylabel('Value')
#         plt.grid(True)
#         plt.legend()
#         plt.tight_layout()
#         plt.show()    
#         plt.savefig(f'Missing {location}, epochs=200, batch_size=64, callbacks (patience=50), validation_split=0.2.png')
    
    
# current_time = datetime.now()

# print('\n')
# print('\nMean of Similarity combine: ',np.mean(sim_lst_combine))
# print('\nMean of Mean Absoulute Error combine :',np.mean(mae_lst_combine))
# print('\nMean of Root Mean Squared Error combine: ',np.mean(rmse_lst_combine))
# print('\nMean of Fraction of Standard Deviation combine: ',np.mean(fsd_lst_combine)) 
# print('\nMean of R-score combine: ', np.mean(r_lst_combine))
# print('\nMean of the Nash Sutcliffe efficiency (NSE): ', np.mean(nse_lst_combine))
# print('\n')

### epochs=200, batch_size=128, callbacks (patience=50), validation_split=0.2

In [ ]:
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# # tf.debugging.set_log_device_placement(True)

# early_stopping = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)


# for location in range(0,12):
    
#     filename = f'waterlevel_missing_{location}.csv'
#     df = pd.read_csv(filename)
    
#     #### Check size of missing value
#     size_of_gap = df['Waterlevel'].isna().sum()

#     data = df['Waterlevel'].values.tolist()

#     nan_index = None
#     for j, value in enumerate(data):
#         if value != value:  # Check if the value is NaN
#             nan_index = j
#             break
        
#     df_check_before = data[:(3*size_of_gap)+1]
#     df_check_after = data[::-1][:3*size_of_gap+1]
#     df_miss = data[nan_index:nan_index + size_of_gap]

#     last_data = data[:nan_index]
#     first_data = data[nan_index+size_of_gap:][::-1]

#     # check if missing values is in the first 3 x T data original
#     if all(value in df_check_before for value in df_miss):
#         print('\nAll values in df_miss is in the first !!!')

#         first_value_combine = transform_to_multivariate(first_data, size_of_gap)
                
#         df_list = to_df(first_value_combine)
        

#         X = np.array(df_list.iloc[:, :-1])
#         X = scaler.fit_transform(X)
        
#         y = np.array(df_list.iloc[:, -1])
        
#         model = model_combine(X)
#         model.fit(X, y, epochs=200, batch_size=128, callbacks=[early_stopping], validation_split=0.2)

#         data_test = df.values.tolist()[nan_index : nan_index + 2 * size_of_gap][::-1]
#         data_test = np.concatenate(data_test).ravel()   

#         results_first_combine = []
#         for i in range(len(data_test)//2):
#             data_first = data_test[i:i+1+size_of_gap]

#             data_first[size_of_gap] = model.predict(scaler.transform(np.array(data_first[:size_of_gap]).reshape(1,-1)))

#             results_first_combine.append(data_first[size_of_gap])
            
#         ###################################################################
#         print('\n', 'result of combine only (first):')                        #  
#         calculate_metrics_for_combine(results_first_combine,'results_combine (FIRST)')#
#         print('\n')                                                       #
#         ###################################################################
        
#         df = pd.read_csv('Impute_misvalues_hungyen.csv')

#         plt.figure(figsize=(10, 6))
#         plt.plot(df['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap], label='Water Level')
#         plt.plot(results_first_combine, label='Predicted Value', linestyle='-')
#         plt.title(f'Water Level vs Predicted Value, mising {location}, epochs=200, batch_size=128, callbacks (patience=50), validation_split=0.2')
#         plt.xlabel('Index')
#         plt.ylabel('Value')
#         plt.grid(True)
#         plt.legend()
#         plt.tight_layout()
#         plt.savefig(f'Missing {location}, epochs=200, batch_size=128, callbacks (patience=50), validation_split=0.2.png')
#         plt.show()
        
#     elif all(value in df_check_after for value in df_miss):
#         print('\nAll values in df_miss is in the last !!!')
        
#         last_value_combine = transform_to_multivariate(last_data, size_of_gap)
                
#         df_list = to_df(last_value_combine)

#         X = np.array(df_list.iloc[:, :-1])
#         X = scaler.fit_transform(X)
        
#         y = np.array(df_list.iloc[:, -1])
        
#         model = model_combine(X)
#         model.fit(X, y, epochs=200, batch_size=128, callbacks=[early_stopping], validation_split=0.2)

#         data_test = df.values.tolist()[nan_index - size_of_gap : nan_index + size_of_gap]
#         data_test = np.concatenate(data_test).ravel()   

#         results_last_combine = []
#         for i in range(len(data_test)//2):
#             data_last = data_test[i:i+1+size_of_gap]

#             data_last[size_of_gap] = model.predict(scaler.transform(np.array(data_last[:size_of_gap]).reshape(1,-1)))

#             results_last_combine.append(data_last[size_of_gap])
            
#         #################################################################
#         print('\n', 'result of combine only (last):')                       #
#         calculate_metrics_for_combine(results_last_combine,'results_combine (LAST)')#
#         print('\n')                                                     #
#         # #################################################################
        
#         df = pd.read_csv('Impute_misvalues_hungyen.csv')

#         plt.figure(figsize=(10, 6))
#         plt.plot(df['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap], label='Water Level')
#         plt.plot(results_last_combine, label='Predicted Value', linestyle='-')
#         plt.title(f'Water Level vs Predicted Value, mising {location}, epochs=200, batch_size=128, callbacks (patience=50), validation_split=0.2')
#         plt.xlabel('Index')
#         plt.ylabel('Value')
#         plt.grid(True)
#         plt.legend()
#         plt.tight_layout()
#         plt.savefig(f'Missing {location}, epochs=200, batch_size=128, callbacks (patience=50), validation_split=0.2.png')
#         plt.show()
        
#     else:
#         Da = data[nan_index + size_of_gap:][::-1]

#         MDa = transform_to_multivariate(Da, size_of_gap)

#         df_MDa = to_df(MDa)

#         X_MDa_train = np.array(df_MDa.iloc[:, :-1])
#         X_MDa_train = scaler.fit_transform(X_MDa_train)
        
#         y_MDa_train = np.array(df_MDa.iloc[:, -1])

#         model_MDa = model_combine(X_MDa_train)
#         model_MDa.fit(X_MDa_train, y_MDa_train, epochs=200, batch_size=128, callbacks=[early_stopping], validation_split=0.2)
        
#         data_test_after = df.values.tolist()[nan_index:nan_index + 2 * size_of_gap ][::-1]
#         data_test_after = np.concatenate(data_test_after).ravel()   

#         value_lst_after = []
#         for j in range(len(data_test_after)//2):
#             data_after = data_test_after[j:j+1+size_of_gap]
            
#             data_after[size_of_gap] = model_MDa.predict(scaler.transform(np.array(data_after[:size_of_gap]).reshape(1,-1)))
            
#             value_lst_after.append(data_after[size_of_gap])

#         Db = data[:nan_index]

#         MDb = transform_to_multivariate(Db, size_of_gap)  

#         df_MDb = to_df(MDb)

#         X_MDb_train = np.array(df_MDb.iloc[:, :-1])
#         X_MDb_train = scaler.fit_transform(X_MDb_train)
        
#         y_MDb_train = np.array(df_MDb.iloc[:, -1])

#         model_MDb = model_combine(X_MDb_train)
#         model_MDb.fit(X_MDb_train, y_MDb_train, epochs=200, batch_size=128, callbacks=[early_stopping], validation_split=0.2)

#         data_test_before = df.values.tolist()[nan_index - size_of_gap : nan_index + size_of_gap]
#         data_test_before = np.concatenate(data_test_before).ravel()   

#         value_lst_before = []
#         for i in range(len(data_test_before)//2):
#             data_before = data_test_before[i:i+1+size_of_gap]
            
#             data_before[size_of_gap] = model_MDb.predict(scaler.transform(np.array(data_before[:size_of_gap]).reshape(1,-1)))

#             value_lst_before.append(data_before[size_of_gap])

#         #############################################################################
#         print('\n', 'result of combine only: ')                                         #
#         results_combine =  [(x + y)/2 for x,y in zip(value_lst_before, value_lst_after)]#
#         calculate_metrics_for_combine(results_combine,'results_combine (MIDDLE)')               #
#         print('\n')                                                                 #
#         #############################################################################
        
#         df = pd.read_csv('Impute_misvalues_hungyen.csv')

#         plt.figure(figsize=(10, 6))
#         plt.plot(df['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap], label='Water Level')
#         plt.plot(results_combine, label='Predicted Value', linestyle='-')
#         plt.title(f'Water Level vs Predicted Value, mising {location}, epochs=200, batch_size=128, callbacks (patience=50), validation_split=0.2')
#         plt.xlabel('Index')
#         plt.ylabel('Value')
#         plt.grid(True)
#         plt.legend()
#         plt.tight_layout()
#         plt.show()    
#         plt.savefig(f'Missing {location}, epochs=200, batch_size=128, callbacks (patience=50), validation_split=0.2.png')
    
    
# current_time = datetime.now()

# print('\n')
# print('\nMean of Similarity combine: ',np.mean(sim_lst_combine))
# print('\nMean of Mean Absoulute Error combine :',np.mean(mae_lst_combine))
# print('\nMean of Root Mean Squared Error combine: ',np.mean(rmse_lst_combine))
# print('\nMean of Fraction of Standard Deviation combine: ',np.mean(fsd_lst_combine)) 
# print('\nMean of R-score combine: ', np.mean(r_lst_combine))
# print('\nMean of the Nash Sutcliffe efficiency (NSE): ', np.mean(nse_lst_combine))
# print('\n')

### epochs=200, batch_size=256, callbacks (patience=50), validation_split=0.2

In [ ]:
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# # tf.debugging.set_log_device_placement(True)

# early_stopping = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)


# for location in range(0,12):
    
#     filename = f'waterlevel_missing_{location}.csv'
#     df = pd.read_csv(filename)
    
#     #### Check size of missing value
#     size_of_gap = df['Waterlevel'].isna().sum()

#     data = df['Waterlevel'].values.tolist()

#     nan_index = None
#     for j, value in enumerate(data):
#         if value != value:  # Check if the value is NaN
#             nan_index = j
#             break
        
#     df_check_before = data[:(3*size_of_gap)+1]
#     df_check_after = data[::-1][:3*size_of_gap+1]
#     df_miss = data[nan_index:nan_index + size_of_gap]

#     last_data = data[:nan_index]
#     first_data = data[nan_index+size_of_gap:][::-1]

#     # check if missing values is in the first 3 x T data original
#     if all(value in df_check_before for value in df_miss):
#         print('\nAll values in df_miss is in the first !!!')

#         first_value_combine = transform_to_multivariate(first_data, size_of_gap)
                
#         df_list = to_df(first_value_combine)
        

#         X = np.array(df_list.iloc[:, :-1])
#         X = scaler.fit_transform(X)
        
#         y = np.array(df_list.iloc[:, -1])
        
#         model = model_combine(X)
#         model.fit(X, y, epochs=200, batch_size=256, callbacks=[early_stopping], validation_split=0.2)

#         data_test = df.values.tolist()[nan_index : nan_index + 2 * size_of_gap][::-1]
#         data_test = np.concatenate(data_test).ravel()   

#         results_first_combine = []
#         for i in range(len(data_test)//2):
#             data_first = data_test[i:i+1+size_of_gap]

#             data_first[size_of_gap] = model.predict(scaler.transform(np.array(data_first[:size_of_gap]).reshape(1,-1)))

#             results_first_combine.append(data_first[size_of_gap])
            
#         ###################################################################
#         print('\n', 'result of combine only (first):')                        #  
#         calculate_metrics_for_combine(results_first_combine,'results_combine (FIRST)')#
#         print('\n')                                                       #
#         ###################################################################
        
#         df = pd.read_csv('Impute_misvalues_hungyen.csv')

#         plt.figure(figsize=(10, 6))
#         plt.plot(df['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap], label='Water Level')
#         plt.plot(results_first_combine, label='Predicted Value', linestyle='-')
#         plt.title(f'Water Level vs Predicted Value, mising {location}, epochs=200, batch_size=256, callbacks (patience=50), validation_split=0.2')
#         plt.xlabel('Index')
#         plt.ylabel('Value')
#         plt.grid(True)
#         plt.legend()
#         plt.tight_layout()
#         plt.savefig(f'Missing {location}, epochs=200, batch_size=256, callbacks (patience=50), validation_split=0.2.png')
#         plt.show()
        
#     elif all(value in df_check_after for value in df_miss):
#         print('\nAll values in df_miss is in the last !!!')
        
#         last_value_combine = transform_to_multivariate(last_data, size_of_gap)
                
#         df_list = to_df(last_value_combine)

#         X = np.array(df_list.iloc[:, :-1])
#         X = scaler.fit_transform(X)
        
#         y = np.array(df_list.iloc[:, -1])
        
#         model = model_combine(X)
#         model.fit(X, y, epochs=200, batch_size=256, callbacks=[early_stopping], validation_split=0.2)

#         data_test = df.values.tolist()[nan_index - size_of_gap : nan_index + size_of_gap]
#         data_test = np.concatenate(data_test).ravel()   

#         results_last_combine = []
#         for i in range(len(data_test)//2):
#             data_last = data_test[i:i+1+size_of_gap]

#             data_last[size_of_gap] = model.predict(scaler.transform(np.array(data_last[:size_of_gap]).reshape(1,-1)))

#             results_last_combine.append(data_last[size_of_gap])
            
#         #################################################################
#         print('\n', 'result of combine only (last):')                       #
#         calculate_metrics_for_combine(results_last_combine,'results_combine (LAST)')#
#         print('\n')                                                     #
#         # #################################################################
        
#         df = pd.read_csv('Impute_misvalues_hungyen.csv')

#         plt.figure(figsize=(10, 6))
#         plt.plot(df['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap], label='Water Level')
#         plt.plot(results_last_combine, label='Predicted Value', linestyle='-')
#         plt.title(f'Water Level vs Predicted Value, mising {location}, epochs=200, batch_size=256, callbacks (patience=50), validation_split=0.2')
#         plt.xlabel('Index')
#         plt.ylabel('Value')
#         plt.grid(True)
#         plt.legend()
#         plt.tight_layout()
#         plt.savefig(f'Missing {location}, epochs=200, batch_size=256, callbacks (patience=50), validation_split=0.2.png')
#         plt.show()
        
#     else:
#         Da = data[nan_index + size_of_gap:][::-1]

#         MDa = transform_to_multivariate(Da, size_of_gap)

#         df_MDa = to_df(MDa)

#         X_MDa_train = np.array(df_MDa.iloc[:, :-1])
#         X_MDa_train = scaler.fit_transform(X_MDa_train)
        
#         y_MDa_train = np.array(df_MDa.iloc[:, -1])

#         model_MDa = model_combine(X_MDa_train)
#         model_MDa.fit(X_MDa_train, y_MDa_train, epochs=200, batch_size=256, callbacks=[early_stopping], validation_split=0.2)
        
#         data_test_after = df.values.tolist()[nan_index:nan_index + 2 * size_of_gap ][::-1]
#         data_test_after = np.concatenate(data_test_after).ravel()   

#         value_lst_after = []
#         for j in range(len(data_test_after)//2):
#             data_after = data_test_after[j:j+1+size_of_gap]
            
#             data_after[size_of_gap] = model_MDa.predict(scaler.transform(np.array(data_after[:size_of_gap]).reshape(1,-1)))
            
#             value_lst_after.append(data_after[size_of_gap])

#         Db = data[:nan_index]

#         MDb = transform_to_multivariate(Db, size_of_gap)  

#         df_MDb = to_df(MDb)

#         X_MDb_train = np.array(df_MDb.iloc[:, :-1])
#         X_MDb_train = scaler.fit_transform(X_MDb_train)
        
#         y_MDb_train = np.array(df_MDb.iloc[:, -1])

#         model_MDb = model_combine(X_MDb_train)
#         model_MDb.fit(X_MDb_train, y_MDb_train, epochs=200, batch_size=256, callbacks=[early_stopping], validation_split=0.2)

#         data_test_before = df.values.tolist()[nan_index - size_of_gap : nan_index + size_of_gap]
#         data_test_before = np.concatenate(data_test_before).ravel()   

#         value_lst_before = []
#         for i in range(len(data_test_before)//2):
#             data_before = data_test_before[i:i+1+size_of_gap]
            
#             data_before[size_of_gap] = model_MDb.predict(scaler.transform(np.array(data_before[:size_of_gap]).reshape(1,-1)))

#             value_lst_before.append(data_before[size_of_gap])

#         #############################################################################
#         print('\n', 'result of combine only: ')                                         #
#         results_combine =  [(x + y)/2 for x,y in zip(value_lst_before, value_lst_after)]#
#         calculate_metrics_for_combine(results_combine,'results_combine (MIDDLE)')               #
#         print('\n')                                                                 #
#         #############################################################################
        
#         df = pd.read_csv('Impute_misvalues_hungyen.csv')

#         plt.figure(figsize=(10, 6))
#         plt.plot(df['Waterlevel'].values.tolist()[nan_index:nan_index+size_of_gap], label='Water Level')
#         plt.plot(results_combine, label='Predicted Value', linestyle='-')
#         plt.title(f'Water Level vs Predicted Value, mising {location}, epochs=200, batch_size=256, callbacks (patience=50), validation_split=0.2')
#         plt.xlabel('Index')
#         plt.ylabel('Value')
#         plt.grid(True)
#         plt.legend()
#         plt.tight_layout()
#         plt.show()    
#         plt.savefig(f'Missing {location}, epochs=200, batch_size=256, callbacks (patience=50), validation_split=0.2.png')
    
    
# current_time = datetime.now()

# print('\n')
# print('\nMean of Similarity combine: ',np.mean(sim_lst_combine))
# print('\nMean of Mean Absoulute Error combine :',np.mean(mae_lst_combine))
# print('\nMean of Root Mean Squared Error combine: ',np.mean(rmse_lst_combine))
# print('\nMean of Fraction of Standard Deviation combine: ',np.mean(fsd_lst_combine)) 
# print('\nMean of R-score combine: ', np.mean(r_lst_combine))
# print('\nMean of the Nash Sutcliffe efficiency (NSE): ', np.mean(nse_lst_combine))
# print('\n')